In [4]:
import os
import pandas as pd
from copy import deepcopy
from os import listdir
from os.path import isfile, join
import string
import numpy as np
import re

#  OMAP: A New Benchmark Dataset



Leggere quanto scritto nel file "OMAP.pdf"

è costituito da questi file

omop_synthea_data.xlsx
omop_mimic_data.xlsx
omop_cms_data.xlsx

dal quale ho ricavato le 4 tabelle che sono state usate in MedCat

TabellaOMOP.csv
TabellaCMS.csv
TabellaSYNTHEA.csv
TabellaMIMIC.csv




In [9]:
Synthea=pd.read_excel('files\omop_synthea_data.xlsx')
Csm=pd.read_excel('files\omop_cms_data.xlsx')
Mimic=pd.read_excel('files\omop_mimic_data.xlsx')

<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:2: SyntaxWarning: invalid escape sequence '\o'
<>:3: SyntaxWarning: invalid escape sequence '\o'
<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:2: SyntaxWarning: invalid escape sequence '\o'
<>:3: SyntaxWarning: invalid escape sequence '\o'
C:\Users\gabry\AppData\Local\Temp\ipykernel_3164\717127518.py:1: SyntaxWarning: invalid escape sequence '\o'
  Synthea=pd.read_excel('files\omop_synthea_data.xlsx')
C:\Users\gabry\AppData\Local\Temp\ipykernel_3164\717127518.py:2: SyntaxWarning: invalid escape sequence '\o'
  Csm=pd.read_excel('files\omop_cms_data.xlsx')
C:\Users\gabry\AppData\Local\Temp\ipykernel_3164\717127518.py:3: SyntaxWarning: invalid escape sequence '\o'
  Mimic=pd.read_excel('files\omop_mimic_data.xlsx')


In [ ]:
len(Synthea[['omop']].drop_duplicates()), len(Csm[['omop']].drop_duplicates()), len(Synthea[['table']].drop_duplicates()), len(Csm[['table']].drop_duplicates()),


(267, 267, 111, 96)

In [ ]:
Synthea['SOURCE']='Synthea'
Csm['SOURCE']='Csm'
Mimic['SOURCE']='Mimic'
df = pd.concat([Synthea, Csm,Mimic])

In [ ]:
pd.set_option('display.max_colwidth', None)
TABELLE_ATTRIBUTI_OMOP=df[['omop','des1','d1','d2']].drop_duplicates()
TABELLE_ATTRIBUTI_OMOP[['Tabella', 'Attributo']] = TABELLE_ATTRIBUTI_OMOP['omop'].str.split('-', expand=True)

TABELLE_ATTRIBUTI_OMOP.sample()
TABELLE_ATTRIBUTI_OMOP['SOURCE']='OMOP'

In [ ]:
## per avere lista di tabelle e attributi OMOP
TABELLE_ATTRIBUTI_OMOP.groupby('Tabella')['Attributo'].agg(['count', list]).reset_index()

,Tabella,count,list
0,care_site,6,"[care_site_id, care_site_name, place_of_service_concept_id, location_id, care_site_source_value, place_of_service_source_value]"
1,cdm_source,10,"[cdm_source_name, cdm_source_abbreviation, cdm_holder, source_description, source_documentation_reference, cdm_etl_reference, source_release_date, cdm_release_date, cdm_version, vocabulary_version]"
2,cohort,4,"[cohort_definition_id, subject_id, cohort_start_date, cohort_end_date]"
3,cohort_definition,7,"[cohort_definition_id, cohort_definition_name, cohort_definition_description, definition_type_concept_id, cohort_definition_syntax, subject_concept_id, cohort_initiation_date]"
4,condition_occurrence,16,"[condition_occurrence_id, person_id, condition_concept_id, condition_start_date, condition_start_datetime, condition_end_date, condition_end_datetime, condition_type_concept_id, condition_status_concept_id, stop_reason, provider_id, visit_occurrence_id, visit_detail_id, condition_source_value, condition_source_concept_id, condition_status_source_value]"
5,death,7,"[person_id, death_date, death_datetime, death_type_concept_id, cause_concept_id, cause_source_value, cause_source_concept_id]"
6,device_exposure,15,"[device_exposure_id, person_id, device_concept_id, device_exposure_start_date, device_exposure_start_datetime, device_exposure_end_date, device_exposure_end_datetime, device_type_concept_id, unique_device_id, quantity, provider_id, visit_occurrence_id, visit_detail_id, device_source_value, device_source_concept_id]"
7,drug_exposure,23,"[drug_exposure_id, person_id, drug_concept_id, drug_exposure_start_date, drug_exposure_start_datetime, drug_exposure_end_date, drug_exposure_end_datetime, verbatim_end_date, drug_type_concept_id, stop_reason, refills, quantity, days_supply, sig, route_concept_id, lot_number, provider_id, visit_occurrence_id, visit_detail_id, drug_source_value, drug_source_concept_id, route_source_value, dose_unit_source_value]"
8,fact_relationship,5,"[domain_concept_id_1, fact_id_1, domain_concept_id_2, fact_id_2, relationship_concept_id]"
9,location,8,"[location_id, address_1, address_2, city, state, zip, county, location_source_value]"


In [ ]:
TABELLE_ATTRIBUTI_LOCALI=df[['SOURCE','table','des2','d3','d4']].drop_duplicates()
TABELLE_ATTRIBUTI_LOCALI.sample()

,SOURCE,table,des2,d3,d4
64,Synthea,observations-type,patient observations including vital signs and lab reports.;the datatype of value text or numeric.,patient observations including vital signs and lab reports.,the datatype of value text or numeric.


In [ ]:
SCHEMA_DI_OMOP=TABELLE_ATTRIBUTI_OMOP[['SOURCE', 'omop','Tabella', 'Attributo']]

SCHEMA_DI_OMOP.columns=['SOURCE', 'AttributoNomeCompleto','Tabella', 'Attributo']
SCHEMA_DI_OMOP['AttributoNomeCompleto'] = SCHEMA_DI_OMOP['AttributoNomeCompleto'].apply(lambda x: x.replace('-', '_', 1))
SCHEMA_DI_OMOP.head()

<ipython-input-11-41ab3ea3be03>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SCHEMA_DI_OMOP['AttributoNomeCompleto'] = SCHEMA_DI_OMOP['AttributoNomeCompleto'].apply(lambda x: x.replace('-', '_', 1))


,SOURCE,AttributoNomeCompleto,Tabella,Attributo
0,OMOP,person_person_id,person,person_id
111,OMOP,person_gender_concept_id,person,gender_concept_id
222,OMOP,person_year_of_birth,person,year_of_birth
333,OMOP,person_month_of_birth,person,month_of_birth
444,OMOP,person_day_of_birth,person,day_of_birth


In [ ]:
TABELLE_ATTRIBUTI_LOCALI.columns
TABELLE_ATTRIBUTI_LOCALI

,SOURCE,table,des2,d3,d4
0,Synthea,allergies-start,patient allergy data.;the date the allergy was diagnosed.,patient allergy data.,the date the allergy was diagnosed.
1,Synthea,allergies-stop,"patient allergy data.;the date the allergy ended, if applicable.",patient allergy data.,"the date the allergy ended, if applicable."
2,Synthea,allergies-patient,patient allergy data.;foreign key to the patient.,patient allergy data.,foreign key to the patient.
3,Synthea,allergies-encounter,patient allergy data.;foreign key to the encounter when the allergy was diagnosed.,patient allergy data.,foreign key to the encounter when the allergy was diagnosed.
4,Synthea,allergies-code,patient allergy data.;allergy code from snomed-ct,patient allergy data.,allergy code from snomed-ct
...,...,...,...,...,...
235,Mimic,transfers-curr_wardid,transfers are physical locations for patients throughout their hospital stay.;curr_wardid contains the current ward in which the patient stayed.,transfers are physical locations for patients throughout their hospital stay.,curr_wardid contains the current ward in which the patient stayed.
236,Mimic,transfers-intime,transfers are physical locations for patients throughout their hospital stay.;intime provides the date and time the patient was transferred into the current care unit from the previous care unit.,transfers are physical locations for patients throughout their hospital stay.,intime provides the date and time the patient was transferred into the current care unit from the previous care unit.
237,Mimic,transfers-outtime,transfers are physical locations for patients throughout their hospital stay.;outtime provides the date and time the patient was transferred out of the current care unit.,transfers are physical locations for patients throughout their hospital stay.,outtime provides the date and time the patient was transferred out of the current care unit.
238,Mimic,transfers-los,"transfers are physical locations for patients throughout their hospital stay.;los is the length of stay for the patient for the given ward stay, which may be within or outside of the icu.",transfers are physical locations for patients throughout their hospital stay.,"los is the length of stay for the patient for the given ward stay, which may be within or outside of the icu."


In [ ]:
TABELLE_ATTRIBUTI_LOCALI[['Tabella', 'Attributo']] = TABELLE_ATTRIBUTI_LOCALI['table'].str.split('-', expand=True)


SCHEMA_Sorgenti_LOCALI=TABELLE_ATTRIBUTI_LOCALI[['SOURCE', 'table', 'Tabella', 'Attributo']]
SCHEMA_Sorgenti_LOCALI.columns=['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo']
SCHEMA_Sorgenti_LOCALI['AttributoNomeCompleto'] = SCHEMA_Sorgenti_LOCALI['AttributoNomeCompleto'].apply(lambda x: x.replace('-', '_', 1))
SCHEMA_Sorgenti_LOCALI

<ipython-input-13-03e344f67b7a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SCHEMA_Sorgenti_LOCALI['AttributoNomeCompleto'] = SCHEMA_Sorgenti_LOCALI['AttributoNomeCompleto'].apply(lambda x: x.replace('-', '_', 1))


,SOURCE,AttributoNomeCompleto,Tabella,Attributo
0,Synthea,allergies_start,allergies,start
1,Synthea,allergies_stop,allergies,stop
2,Synthea,allergies_patient,allergies,patient
3,Synthea,allergies_encounter,allergies,encounter
4,Synthea,allergies_code,allergies,code
...,...,...,...,...
235,Mimic,transfers_curr_wardid,transfers,curr_wardid
236,Mimic,transfers_intime,transfers,intime
237,Mimic,transfers_outtime,transfers,outtime
238,Mimic,transfers_los,transfers,los


In [ ]:
TABELLE_ATTRIBUTI_OMOP['SOURCE']='OMOP'

In [ ]:
TABELLE_ATTRIBUTI_LOCALI.columns

Index(['SOURCE', 'table', 'des2', 'd3', 'd4', 'Tabella', 'Attributo'], dtype='object')

In [ ]:
SCHEMA_DI_OMOP.columns

Index(['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo'], dtype='object')

In [ ]:
TuttiGliSchemi=pd.concat([SCHEMA_Sorgenti_LOCALI,SCHEMA_DI_OMOP])
TuttiGliSchemi.columns

TuttiGliSchemi.groupby('SOURCE')['Tabella'].nunique().reset_index(name='Num_Tabelle')

,SOURCE,Num_Tabelle
0,Csm,5
1,Mimic,25
2,OMOP,21
3,Synthea,12


In [ ]:
TuttiGliSchemi.groupby(['SOURCE','Tabella'])['Attributo'].nunique().reset_index(name='Num_Attributi')


,SOURCE,Tabella,Num_Attributi
0,Csm,beneficiarysummary,32
1,Csm,carrierclaims,15
2,Csm,inpatientclaims,23
3,Csm,outpatientclaims,18
4,Csm,prescriptiondrugevents,8
...,...,...,...
58,Synthea,observations,8
59,Synthea,organizations,8
60,Synthea,patients,20
61,Synthea,procedures,8


In [ ]:
TuttiGliSchemi

,SOURCE,AttributoNomeCompleto,Tabella,Attributo
0,Synthea,allergies_start,allergies,start
1,Synthea,allergies_stop,allergies,stop
2,Synthea,allergies_patient,allergies,patient
3,Synthea,allergies_encounter,allergies,encounter
4,Synthea,allergies_code,allergies,code
...,...,...,...,...
29082,OMOP,provider_provider_source_value,provider,provider_source_value
29193,OMOP,provider_specialty_source_value,provider,specialty_source_value
29304,OMOP,provider_specialty_source_concept_id,provider,specialty_source_concept_id
29415,OMOP,provider_gender_source_value,provider,gender_source_value


## Match

I matches sono quelli segnati con
label==1

In [ ]:

df=df.query('label==1')


GMT=df[['SOURCE','omop','table']]
GMT.columns=['SOURCE','GAT','LAT']
GMT.sample()

,SOURCE,GAT,LAT
61001,Mimic,provider-provider_id,caregivers-mimic_id


In [ ]:
def to_GMT(MT:pd.DataFrame):
  GMT = pd.DataFrame(columns=['GAT','SOURCE', 'LAT','SLAT'])

  for x in MT.index:
    for y in MT.columns:
      for z in MT.loc[x][y]:
        GMT.loc[len(GMT)]=[x,y,z,str(y)+ '_' + z]
  return GMT

def to_GMM(GMTA:pd.DataFrame):
    df = GMTA.groupby(['GAT','SOURCE'])['LAT'].agg(list).unstack('SOURCE')
    for c in df.columns:
        df.loc[df[c].isnull(), [c]] = df.loc[df[c].isnull(), c].apply(lambda x: [])
    return df

# nuova funzione
def to_GMM(GMTA: pd.DataFrame):
    df = GMTA.groupby(['GAT', 'SOURCE'])['LAT'].agg(list).unstack('SOURCE')

    for c in df.columns:
        df[c] = df[c].apply(lambda x: [] if isinstance(x, float) and np.isnan(x) else x)

    return df

# per generare la Local Attribute Table
def generaLAT(Sources):
  LAT = pd.DataFrame(columns=['SOURCE', 'LAT', 'SLAT'])
  for x in Sources:
    for y in Sources[x].columns:
      LAT.loc[len(LAT)]=[str(x),str(y), str(x)+'_'+str(y)]
  return LAT

def sim_table(TableA:pd.DataFrame, TableB:pd.DataFrame):
    A = pd.DataFrame({"A": TableA.columns})
    B = pd.DataFrame({"B": TableB.columns})
    S = A.merge(B, how='cross')
    return S

def random_sim_table(TableA:pd.DataFrame, TableB:pd.DataFrame):
    S = sim_table(TableA, TableB)
    S["sim"] = np.random.rand(len(S))
    return S

def to_sim_table(SimMatrix:pd.DataFrame):
    return SimMatrix.stack().reset_index(name="sim")

def to_sim_matrix(SimTable:pd.DataFrame):
    return SimTable.pivot(index="A", columns="B", values="sim") \
              .rename_axis(None, axis=1).rename_axis(None, axis=0)

def sim_table(TableA:pd.DataFrame, TableB:pd.DataFrame):
    A = pd.DataFrame({"A": TableA.columns})
    B = pd.DataFrame({"B": TableB.columns})
    S = A.merge(B, how='cross')
    return S

def random_sim_table(TableA:pd.DataFrame, TableB:pd.DataFrame):
    S = sim_table(TableA, TableB)
    S["sim"] = np.random.rand(len(S))
    return S

def to_sim_table(SimMatrix:pd.DataFrame):
    return SimMatrix.stack().reset_index(name="sim")

def to_sim_matrix(SimTable:pd.DataFrame):
    return SimTable.pivot(index="A", columns="B", values="sim") \
              .rename_axis(None, axis=1).rename_axis(None, axis=0)

def string_preprocess(s:str, char:str=string.punctuation, word:list=[]):
    if type(s) is str:
        s = s.lower()
        for c in char:
            s = s.replace(c, " ")
        for w in word:
            s = s.replace(w, " ")
    else:
        s = str(s)
    s = re.sub(" +", " ", s)
    return s.strip()

In [ ]:
to_GMM(GMT)

SOURCE,Csm,Mimic,Synthea
GAT,,,
care_site-care_site_name,[],[transfers-curr_wardid],[]
care_site-care_site_source_value,"[inpatientclaims-prvdr_num, outpatientclaims-prvdr_num, carrierclaims-tax_num]",[],[]
cohort-cohort_definition_id,"[beneficiarysummary-bene_esrd_ind, beneficiarysummary-sp_alzhdmta, beneficiarysummary-sp_chf, beneficiarysummary-sp_chrnkidn, beneficiarysummary-sp_cncr, beneficiarysummary-sp_copd, beneficiarysummary-sp_depressn, beneficiarysummary-sp_diabetes, beneficiarysummary-sp_ischmcht, beneficiarysummary-sp_osteoprs, beneficiarysummary-sp_ra_oa, beneficiarysummary-sp_strketia]",[],[]
cohort-subject_id,[beneficiarysummary-desynpuf_id],[],[]
cohort_definition-cohort_definition_name,"[beneficiarysummary-bene_esrd_ind, beneficiarysummary-sp_alzhdmta, beneficiarysummary-sp_chf, beneficiarysummary-sp_chrnkidn, beneficiarysummary-sp_cncr, beneficiarysummary-sp_copd, beneficiarysummary-sp_depressn, beneficiarysummary-sp_diabetes, beneficiarysummary-sp_ischmcht, beneficiarysummary-sp_osteoprs, beneficiarysummary-sp_ra_oa, beneficiarysummary-sp_strketia]",[],[]
...,...,...,...
visit_occurrence-visit_end_datetime,[],[admissions-dischtime],[encounters-stop]
visit_occurrence-visit_occurrence_id,[],[admissions-mimic_id],[]
visit_occurrence-visit_source_value,"[inpatientclaims-clm_id, outpatientclaims-clm_id, carrierclaims-clm_id]",[admissions-admission_type],[encounters-encounterclass]


In [ ]:
GMT_person=GMT[GMT['GAT'].str.contains(f'^{"person"}-')]
to_GMM(GMT_person)

SOURCE,Csm,Mimic,Synthea
GAT,,,
person-birth_datetime,[],[patients-dob],[patients-birthdate]
person-day_of_birth,[beneficiarysummary-bene_birth_dt],[patients-dob],[patients-birthdate]
person-death_datetime,[],[patients-dod],[patients-deathdate]
person-ethnicity_concept_id,[beneficiarysummary-bene_race_cd],[],"[patients-race, patients-ethnicity]"
person-ethnicity_source_value,[beneficiarysummary-bene_race_cd],[],[patients-ethnicity]
person-gender_concept_id,[beneficiarysummary-bene_sex_ident_cd],[patients-gender],[patients-gender]
person-gender_source_concept_id,[beneficiarysummary-bene_sex_ident_cd],[],[]
person-gender_source_value,[beneficiarysummary-bene_sex_ident_cd],[patients-gender],[patients-gender]
person-location_id,"[beneficiarysummary-sp_state_code, beneficiarysummary-bene_county_cd]",[],[]


In [ ]:
to_GMM(GMT[GMT['GAT'].str.contains(f'^{"condition_occurrence"}-')])

SOURCE,Csm,Mimic,Synthea
GAT,,,
condition_occurrence-condition_concept_id,[],[],[conditions-code]
condition_occurrence-condition_end_date,"[inpatientclaims-clm_thru_dt, outpatientclaims-clm_thru_dt, carrierclaims-clm_thru_dt]",[admissions-dischtime],[conditions-stop]
condition_occurrence-condition_end_datetime,[],[admissions-dischtime],[conditions-stop]
condition_occurrence-condition_occurrence_id,[],[diagnoses_icd-mimic_id],[]
condition_occurrence-condition_source_concept_id,[],[],[conditions-code]
condition_occurrence-condition_source_value,"[inpatientclaims-admtng_icd9_dgns_cd, inpatientclaims-icd9_dgns_cd, outpatientclaims-icd9_dgns_cd, carrierclaims-icd9_dgns_cd, carrierclaims-line_icd9_dgns_cd]",[d_icd_diagnoses-icd9_code],[conditions-code]
condition_occurrence-condition_start_date,"[inpatientclaims-clm_admsn_dt, outpatientclaims-clm_from_dt, carrierclaims-clm_from_dt]","[admissions-admittime, admissions-edregtime]",[conditions-start]
condition_occurrence-condition_start_datetime,[],"[admissions-admittime, admissions-edregtime]",[conditions-start]
condition_occurrence-person_id,"[inpatientclaims-desynpuf_id, outpatientclaims-desynpuf_id, carrierclaims-desynpuf_id]",[patients-mimic_id],[conditions-patient]


In [ ]:
to_GMM(GMT[GMT['GAT'].str.contains(f'^{"observation"}-')])

SOURCE,Csm,Mimic,Synthea
GAT,,,
observation-observation_concept_id,[],[],"[allergies-code, conditions-code]"
observation-observation_date,"[inpatientclaims-clm_from_dt, outpatientclaims-clm_from_dt, carrierclaims-clm_from_dt]","[admissions-admittime, datetimeevents-charttime, datetimeevents-value]","[allergies-start, conditions-start]"
observation-observation_datetime,[],"[admissions-admittime, chartevents-charttime, datetimeevents-charttime, datetimeevents-value]","[allergies-start, conditions-start]"
observation-observation_id,[],"[chartevents-mimic_id, datetimeevents-mimic_id, drgcodes-mimic_id]",[]
observation-observation_source_concept_id,[],[],"[allergies-code, conditions-code]"
observation-observation_source_value,"[inpatientclaims-admtng_icd9_dgns_cd, inpatientclaims-clm_drg_cd, inpatientclaims-icd9_dgns_cd, inpatientclaims-icd9_prcdr_cd, outpatientclaims-icd9_dgns_cd, outpatientclaims-hcpcs_cd, carrierclaims-icd9_dgns_cd, carrierclaims-hcpcs_cd, carrierclaims-line_icd9_dgns_cd]",[],"[allergies-code, conditions-code]"
observation-person_id,"[inpatientclaims-desynpuf_id, outpatientclaims-desynpuf_id, carrierclaims-desynpuf_id]",[patients-mimic_id],"[allergies-patient, conditions-patient]"
observation-provider_id,"[inpatientclaims-at_physn_npi, outpatientclaims-at_physn_npi, carrierclaims-prf_physn_npi]",[caregivers-mimic_id],[]
observation-value_as_number,[],[chartevents-valuenum],[]


La GMT mantiene le corrispondenze di ogni singolo schema delle tre sorgenti rispetto allo schema OMOP
(siamo nella situazione top-down)


# MedCAT e l'Annotazione degli Attributi nell'OMOP Benchmark  

L'obiettivo di questo notebook è analizzare i risultati ottenuti dall'applicazione di **MedCAT** all'**OMOP Benchmark**.  

**MedCAT** viene utilizzato per annotare gli attributi di tutte le tabelle appartenenti ai seguenti quattro schemi:  
- **OMOP**  
- **CMS**  
- **SYNTHEA**  
- **MIMIC**  

L'annotazione consiste nell'associare a ciascun attributo un insieme di concetti (codici) provenienti da **UMLS**, ciascuno accompagnato da un valore di **context similarity**.  

L'**AnnotazioneMedCAT** è un'annotazione applicata a un attributo di un dataset. Essa è composta dai seguenti elementi:  

- **`attributo`** → Nome dell'attributo analizzato all'interno del dataset.  
- **`detected_name`** → Termine individuato all'interno di `names`.  
- **`pretty_name`** → Concetto standardizzato UMLS associato al `detected_name`.  
- **`context_similarity`** → Punteggio che indica il grado di affidabilità dell'associazione tra l'attributo e il concetto.  

Un aspetto fondamentale di **MedCAT**, rispetto ad altri sistemi di annotazione, è la restituzione dei concetti UMLS **insieme** al punteggio di **context similarity**.  

Questa caratteristica comporta due conseguenze principali:  
1. **Un singolo attributo può essere associato a più concetti**, ognuno con un proprio valore di **context similarity**.  
2. **Un singolo concetto UMLS può annotare più attributi**, ciascuno con un diverso valore di **context similarity**.  

Questo approccio consente di ottenere un'annotazione più flessibile e dettagliata, permettendo una valutazione più precisa della qualità dell'associazione tra concetti e attributi.  

## sinonimia e omonimia

Prima di tutto bisognerebbe spiegare sinteticamente il significato di source_value 	detected_name,
cioè come si passa dall'uno all'altro.
A me sembra che detected_name sia simile alla forma base o lemma di un termine in wordnet, ma è da verificare


### Forma base di Wordnet

WordNet,  è un ampio database lessicale della lingua inglese. WordNet organizza parole in insiemi di sinonimi (chiamati synsets) che rappresentano un concetto unico. Ogni synset è collegato ad altri tramite varie relazioni semantiche, come sinonimia, antonimia, iponimia (specificità), e meronimia (parte-tutto). È utilizzato in molte applicazioni di elaborazione del linguaggio naturale.

Forma base di un termine (lemma)

La forma base di un termine, o lemma, è la versione standard o “radice” della parola, che si trova in un dizionario. In WordNet, la forma base viene utilizzata per ridurre una parola alle sue radici, eliminando inflessioni grammaticali come tempi verbali, plurali o comparativi.

Ad esempio:
	•	Per i verbi: la forma base è l’infinito senza il “to”.
	•	eating, ate, e eaten → eat
	•	Per i sostantivi: la forma base è il singolare.
	•	children → child, dogs → dog
	•	Per gli aggettivi: la forma base è la versione senza suffissi comparativi o superlativi.
	•	happier o happiest → happy


### corrispondenze tra detected_name e pretty_name

ad un certo detected_name possono corrispondere più pretty_name (questo dovrebbe essere una sinonimia)

ad un certo pretty_name possono corrispondere più detected_name (questo dovrebbe essere una ononimia)

Verifichiamolo

In [ ]:
df=pd.read_csv("files\dataframe_snomed.csv")

In [ ]:
### Annotazione in base ad una soglia di context_similarity

In [ ]:
df_MAX=df[df.context_similarity==1].copy()
df_08=df[df.context_similarity>=.8].copy()

In [ ]:
len(df_MAX), len(df_08), len(df)

(4675, 5039, 5917)

In [ ]:
### Quindi provare con questi (ed eventualmente altri valori di soglia) cosa cambia come match ottenuti

In [ ]:
df.groupby('pretty_name')['detected_name'].nunique().max(),df.groupby('detected_name')['pretty_name'].nunique().max()
# se maggiore di 1 significa che ad un pretty_name corrispondono più e viceversa

(3, 2)

In [ ]:
## Troviamo qualche corrispondenza significativa
# Passo 1: Filtra i pretty_name con almeno 5 detected_name
pretty_name_counts = df.groupby('pretty_name')['detected_name'].nunique()
pretty_names_with_multiple_detected = pretty_name_counts[pretty_name_counts >= 5].index

# Passo 2: Verifica se per ciascun detected_name c'è almeno 5 pretty_name distinti
result = []

for pretty_name in pretty_names_with_multiple_detected:
    detected_names_for_pretty = df[df['pretty_name'] == pretty_name]['detected_name'].unique()

    for detected_name in detected_names_for_pretty:
        pretty_names_for_detected = df[df['detected_name'] == detected_name]['pretty_name'].nunique()

        if pretty_names_for_detected >= 5:
            result.append((pretty_name, detected_name))

# Creazione del DataFrame con i risultati
result_df = pd.DataFrame(result, columns=['pretty_name', 'detected_name'])

# Rimuovi duplicati
result_df = result_df.drop_duplicates()

result_df
result_df[result_df.pretty_name=='Geographic Locations']

# i detected_name che corrispondono allo stesso pretty_name sono dei sinonimi

,pretty_name,detected_name


In [ ]:
# un detected_name che corrispondono a più pretty_name indica ononimia
result_df[result_df.detected_name=='area']

,pretty_name,detected_name
4,Body Regions,area
8,C1254362,area
21,Geographic Locations,area
49,quantitative,area


Verifichiamo che :  
1. **Un singolo attributo può essere associato a più concetti**, ognuno con un proprio valore di **context similarity**.  
2. **Un singolo concetto UMLS può annotare più attributi**, ciascuno con un diverso valore di **context similarity**.  

### df full


In [ ]:
df1=pd.read_csv("files\dataframe_snomed.csv", encoding="ISO-8859-1")
df1.columns

Index(['text', 'entity_id', 'pretty_name', 'cui', 'type_ids', 'types',
       'source_value', 'detected_name', 'acc', 'context_similarity',
       'ontologies', 'names'],
      dtype='object')

In [ ]:
#AnnotazioneMedCAT=df1[['names','detected_name','pretty_name','context_similarity']]

In [ ]:
AnnotazioneMedCAT=df1

In [ ]:
# Creiamo df_result basandoci sul DataFrame di annotazioni MedCAT
df_result = AnnotazioneMedCAT[['names', 'detected_name', 'pretty_name', 'context_similarity']].copy()

# Calcoliamo l'intervallo di context_similarity per ogni pretty_name
df_variation = df_result.groupby('pretty_name').agg(
    min_similarity=('context_similarity', 'min'),
    max_similarity=('context_similarity', 'max'),
    range_similarity=('context_similarity', lambda x: x.max() - x.min())  # Differenza massimo-minimo
)

# Filtriamo solo i pretty_name con una variazione significativa (es. range > 0.5)
threshold = 0.5  # Puoi modificare la soglia a seconda dei dati
df_significant = df_variation[df_variation['range_similarity'] > threshold]

# Recuperiamo i dettagli completi per questi casi
df_detailed = df_result[df_result['pretty_name'].isin(df_significant.index)]

# Ordiniamo per differenza di similarità in ordine decrescente
df_significant_sorted = df_significant.sort_values('range_similarity', ascending=False)

# Mostriamo i 10 casi più significativi
df_significant_sorted.head(10)

,min_similarity,max_similarity,range_similarity
pretty_name,,,
Date (attribute),0.202249,0.990000,0.787751
Source,0.218030,0.990000,0.771970
High (qualifier value),0.241812,1.000000,0.758188
Times,0.213117,0.929999,0.716882
Specimen,0.283667,1.000000,0.716333
Time periods,0.324483,0.990000,0.665517
Measurement procedure,0.352720,1.000000,0.647280
Event,0.219285,0.846575,0.627290
Exposure to (contextual qualifier),0.429760,1.000000,0.570240


### df context similarity = 1


In [ ]:
df1=pd.read_csv("files\dataframe_snomed.csv", encoding="ISO-8859-1")

df_MAX=df1[df1.context_similarity==1].copy()

#AnnotazioneMedCAT_MAX=df_MAX[['names','detected_name','pretty_name','context_similarity']]
AnnotazioneMedCAT_MAX=df_MAX

In [ ]:
AnnotazioneMedCAT_MAX

,names,detected_name,pretty_name,context_similarity
0,beneficiarysummary_bene_birth_dt,date~of~birth,Date of birth,1.0
1,beneficiarysummary_bene_death_dt,date~of~death,Date of death,1.0
2,beneficiarysummary_bene_esrd_ind,end~stage~renal~disease,End-stage renal disease,1.0
3,beneficiarysummary_sp_state_code,state,State,1.0
5,beneficiarysummary_bene_hi_cvrage_tot_mons,of~part,In (attribute),1.0
...,...,...,...,...
5912,providers_state,abbreviation,Abbreviation,1.0
5913,providers_zip,patient,Patient,1.0
5914,providers_zip,street~address,Street address,1.0
5915,providers_utilization,patient,Patient,1.0


In [ ]:
# Creiamo df_result basandoci sul DataFrame di annotazioni MedCAT
df_result = AnnotazioneMedCAT_MAX[['names', 'detected_name', 'pretty_name', 'context_similarity']].copy()

# Calcoliamo l'intervallo di context_similarity per ogni pretty_name
df_variation = df_result.groupby('pretty_name').agg(
    min_similarity=('context_similarity', 'min'),
    max_similarity=('context_similarity', 'max'),
    range_similarity=('context_similarity', lambda x: x.max() - x.min())  # Differenza massimo-minimo
)

df_variation

,min_similarity,max_similarity,range_similarity
pretty_name,,,
Abbreviation,1.0,1.0,0.0
Abnormal,1.0,1.0,0.0
Absent,1.0,1.0,0.0
Accident and Emergency department,1.0,1.0,0.0
Active,1.0,1.0,0.0
...,...,...,...
WAS A,1.0,1.0,0.0
Walking surface of room,1.0,1.0,0.0
colonies,1.0,1.0,0.0


In [ ]:
# Filtriamo solo i pretty_name con una variazione significativa (es. range > 0.5)
threshold = 0.5  # Puoi modificare la soglia a seconda dei dati
df_significant = df_variation[df_variation['range_similarity'] > threshold]

# Recuperiamo i dettagli completi per questi casi
df_detailed = df_result[df_result['pretty_name'].isin(df_significant.index)]

# Ordiniamo per differenza di similarità in ordine decrescente
df_significant_sorted = df_significant.sort_values('range_similarity', ascending=False)

# Mostriamo i 10 casi più significativi
df_significant_sorted

,min_similarity,max_similarity,range_similarity
pretty_name,,,


### df context similarity >= 0.8


In [ ]:
df1=pd.read_csv("files\dataframe_snomed.csv", encoding="ISO-8859-1")

df_08=df1[df1.context_similarity>=.8].copy()

#AnnotazioneMedCAT_08=df_08[['names','detected_name','pretty_name','context_similarity']]
AnnotazioneMedCAT_08=df_08

In [ ]:
# Creiamo df_result basandoci sul DataFrame di annotazioni MedCAT
df_result = AnnotazioneMedCAT_08[['names', 'detected_name', 'pretty_name', 'context_similarity']].copy()

# Calcoliamo l'intervallo di context_similarity per ogni pretty_name
df_variation = df_result.groupby('pretty_name').agg(
    min_similarity=('context_similarity', 'min'),
    max_similarity=('context_similarity', 'max'),
    range_similarity=('context_similarity', lambda x: x.max() - x.min())  # Differenza massimo-minimo
)

# Filtriamo solo i pretty_name con una variazione significativa (es. range > 0.5)
threshold = 0.5  # Puoi modificare la soglia a seconda dei dati
df_significant = df_variation[df_variation['range_similarity'] > threshold]

# Recuperiamo i dettagli completi per questi casi
df_detailed = df_result[df_result['pretty_name'].isin(df_significant.index)]

# Ordiniamo per differenza di similarità in ordine decrescente
df_significant_sorted = df_significant.sort_values('range_similarity', ascending=False)

# Mostriamo i 10 casi più significativi
df_significant_sorted.head(10)

,min_similarity,max_similarity,range_similarity
pretty_name,,,


Ogni *text* (e quindi ogni *att*) ha associato un insieme di *pretty_name*, ciascuno accompagnato da un valore di *context_similarity*. Questi insiemi rappresentano i termini chiave estratti dal testo e il loro relativo peso o rilevanza nel contesto del testo stesso.

### Jaccard (OSS)
La **Jaccard** tra due *text* (ad esempio, *att1* e *att2*) è definita come la misura del grado di sovrapposizione tra i rispettivi insiemi di *pretty_name*. Formalmente:

$$
\text{OSS}(att1, att2) = \frac{|P_{att1} \cap P_{att2}|}{|P_{att1} \cup P_{att2}|}
$$

dove:

- \(P_{att1}\) e \(P_{att2}\) sono gli insiemi di *pretty_name* associati a *att1* e *att2*, rispettivamente.
- \(|P_{att1} \cap P_{att2}|\) rappresenta il numero di *pretty_name* in comune tra i due attributi.
- \(|P_{att1} \cup P_{att2}|\) rappresenta il numero totale di *pretty_name* unici tra i due attributi.

#### Esempio
Supponiamo che:

$$
P_{att1} = \{\text{'A'}, \text{'B'}, \text{'C'}\}, \quad P_{att2} = \{\text{'B'}, \text{'C'}, \text{'D'}\}.
$$

In questo caso:

- \(P_{att1} \cap P_{att2} = \{\text{'B'}, \text{'C'}\}\), quindi due elementi in comune.
- \(P_{att1} \cup P_{att2} = \{\text{'A'}, \text{'B'}, \text{'C'}, \text{'D'}\}\), quindi quattro elementi totali.

Pertanto:

$$
\text{OSS}(att1, att2) = \frac{2}{4} = 0.5
$$

##DF CON TUTTI ELEMENTI

In [ ]:
#AnnotazioneMedCAT.columns= ['attribute', 'detected_name', 'pretty_name', 'context_similarity']

In [ ]:
# aggiungo anche nome tabella e nome attributo
AnnotazioneMedCAT = AnnotazioneMedCAT.copy()

# Separiamo la colonna 'attribute' in base all'ultimo '_', separando in 3 parti
split_columns = AnnotazioneMedCAT['attribute'].str.rsplit('_', n=2, expand=True)

# Usa .loc per assegnare le nuove colonne al DataFrame originale
AnnotazioneMedCAT.loc[:, 'Tabella'] = split_columns[0]
AnnotazioneMedCAT.loc[:, 'Attributo'] = split_columns[1] + '_' + split_columns[2]

# Visualizza il risultato
AnnotazioneMedCAT

,attribute,detected_name,pretty_name,context_similarity,Tabella,Attributo
0,beneficiarysummary_bene_birth_dt,date~of~birth,Date of birth,1.00,beneficiarysummary_bene,birth_dt
1,beneficiarysummary_bene_death_dt,date~of~death,Date of death,1.00,beneficiarysummary_bene,death_dt
2,beneficiarysummary_bene_esrd_ind,end~stage~renal~disease,End-stage renal disease,1.00,beneficiarysummary_bene,esrd_ind
3,beneficiarysummary_sp_state_code,state,State,1.00,beneficiarysummary_sp,state_code
4,beneficiarysummary_bene_hi_cvrage_tot_mons,total,Total,0.99,beneficiarysummary_bene_hi_cvrage,tot_mons
...,...,...,...,...,...,...
5912,providers_state,abbreviation,Abbreviation,1.00,providers,NaN
5913,providers_zip,patient,Patient,1.00,providers,NaN
5914,providers_zip,street~address,Street address,1.00,providers,NaN
5915,providers_utilization,patient,Patient,1.00,providers,NaN


il dataframe GMT ha come columns ['SOURCE', 'GAT', 'LAT']
dove GAT corrisponde ad attribute AnnotazioneMedCAT con l'unica differenza che mentre in GMT
ho un GAT ad esempio beneficiarysummary_bene_birth_dt
in AnnotazioneMedCAT me lo trovo riscritto come beneficiarysummary-bene_birth_dt
cioè il nome della tabella è separato da _ nel primo e da - nel secondo:
quindi prima di fare join devo uniformare

La GMT mantiene le corrispondenze di ogni singolo schema delle tre sorgenti rispetto allo schema OMOP
(siamo nella situazione top-down)


In [ ]:
TuttiGliSchemi.columns

Index(['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo'], dtype='object')

In [ ]:
# Eseguiamo il merge con AnnotazioneMedCAT
merged_df = TuttiGliSchemi.merge(AnnotazioneMedCAT, left_on='AttributoNomeCompleto', right_on='attribute', how='inner')

# Visualizziamo i primi risultati
merged_df.head()

NewAnnotazioneMedCAT=merged_df[['SOURCE', 'AttributoNomeCompleto', 'Tabella_x', 'Attributo_x', 'detected_name','pretty_name', 'context_similarity' ]]

In [ ]:
NewAnnotazioneMedCAT.head()
AnnotazioneMedCAT[AnnotazioneMedCAT['attribute'].isin(NewAnnotazioneMedCAT['AttributoNomeCompleto'])]

KeyError: 'attribute'

In [ ]:
NewAnnotazioneMedCAT[['SOURCE']].drop_duplicates()

,SOURCE
0,Synthea
398,Csm
631,Mimic
1912,OMOP


In [ ]:
NewAnnotazioneMedCAT.columns=['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo',
       'detected_name', 'pretty_name', 'context_similarity']

In [ ]:
# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Mimic']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

In [ ]:
omop_data

,SOURCE,AttributoNomeCompleto,Tabella,Attributo,detected_name,pretty_name,context_similarity
1912,OMOP,person_person_id,person,person_id,person,Person,1.00
1913,OMOP,person_person_id,person,person_id,domain,Concept model domain,1.00
1914,OMOP,person_person_id,person,person_id,contain,Containing,1.00
1915,OMOP,person_person_id,person,person_id,identify,Identified,1.00
1916,OMOP,person_person_id,person,person_id,patient,Patient,1.00
...,...,...,...,...,...,...,...
5921,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,nurse,Professional nurse,1.00
5922,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,midwife,Registered midwife,1.00
5923,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,physical~therapist,Physiotherapist,1.00
5924,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,refer,Patient referral,1.00


In [ ]:
mimic_data

,SOURCE,AttributoNomeCompleto,Tabella,Attributo,detected_name,pretty_name,context_similarity
631,Mimic,admissions_mimic_id,admissions,mimic_id,admission,Hospital admission,1.0
632,Mimic,admissions_mimic_id,admissions,mimic_id,table,"Table, device",1.0
633,Mimic,admissions_mimic_id,admissions,mimic_id,give~information,Informing,1.0
634,Mimic,admissions_mimic_id,admissions,mimic_id,patient,Patient,1.0
635,Mimic,admissions_mimic_id,admissions,mimic_id,admission,Hospital admission,1.0
...,...,...,...,...,...,...,...
1907,Mimic,transfers_mimic_id,transfers,mimic_id,physical,Physical,1.0
1908,Mimic,transfers_mimic_id,transfers,mimic_id,location,Location,1.0
1909,Mimic,transfers_mimic_id,transfers,mimic_id,patient,Patient,1.0
1910,Mimic,transfers_mimic_id,transfers,mimic_id,hospital,Hospital,1.0


In [ ]:
from collections import defaultdict
import pandas as pd

# Definisci una soglia di similarità per OSS
oss_threshold = 0.5  # Puoi modificarla in base alle tue necessità

# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Mimic']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute_omop.get(attribute1, set())
    P2 = pretty_name_by_attribute_mimic.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))
    union = len(P1.union(P2))

    if union == 0:  # Evitiamo la divisione per zero
        return 0.0
    return intersection / union

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA) di OMOP
top_oss_results = defaultdict(list)
no_match_results = []  # Lista per gli attributi che non hanno corrispondenza

# Calcolare OSS tra tutte le combinazioni di attributi di OMOP e MIMIC
for (tableA, attributeA) in pretty_name_by_attribute_omop.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute_mimic.keys():
        oss = calculate_oss((tableA, attributeA), (tableB, attributeB))

        if oss >= oss_threshold:
            oss_list.append(((tableB, attributeB), oss))

    # Se ci sono corrispondenze sopra la soglia, prendi le prime 3
    if oss_list:
        top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]
    else:
        # Se non ci sono corrispondenze sopra la soglia, aggiungi "assenza"
        no_match_results.append((tableA, attributeA, "Assente in MIMIC", "Assente in MIMIC", 0.0))

# Aggiungi i risultati di "assenza" al dizionario finale
df_no_match = pd.DataFrame(no_match_results, columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Convertiamo i risultati delle corrispondenze in un DataFrame
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Unisci i risultati di corrispondenza con quelli di "assenza"
df_final_result = pd.concat([df_top_oss, df_no_match], ignore_index=True)

# Ordina i risultati in base all'OSS in ordine decrescente
df_final_result = df_final_result.sort_values(by='OSS', ascending=False)

# Visualizza i risultati
df_final_result.head(10)  # Mostra i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
36,note,visit_detail_id,inputevents_cv,mimic_id,0.666667
13,cohort_definition,definition_type_concept_id,d_labitems,mimic_id,0.666667
33,note,provider_id,inputevents_cv,mimic_id,0.666667
34,note,provider_id,inputevents_mv,mimic_id,0.666667
35,note,provider_id,outputevents,mimic_id,0.666667
37,note,visit_detail_id,inputevents_mv,mimic_id,0.666667
20,note,note_date,outputevents,mimic_id,0.666667
19,note,note_date,inputevents_mv,mimic_id,0.666667
18,note,note_date,inputevents_cv,mimic_id,0.666667
38,note,visit_detail_id,outputevents,mimic_id,0.666667


In [ ]:
from collections import defaultdict
import pandas as pd

# Definisci una soglia di similarità per OSS
oss_threshold = 0.1  # Puoi modificarla in base alle tue necessità

# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Synthea']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute_omop.get(attribute1, set())
    P2 = pretty_name_by_attribute_mimic.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))
    union = len(P1.union(P2))

    if union == 0:  # Evitiamo la divisione per zero
        return 0.0
    return intersection / union

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA) di OMOP
top_oss_results = defaultdict(list)

# Calcolare OSS tra tutte le combinazioni di attributi di OMOP e MIMIC
for (tableA, attributeA) in pretty_name_by_attribute_omop.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute_mimic.keys():
        oss = calculate_oss((tableA, attributeA), (tableB, attributeB))

        if oss >= oss_threshold:
            oss_list.append(((tableB, attributeB), oss))

    # Se ci sono corrispondenze sopra la soglia, prendi le prime 3
    if oss_list:
        top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]

# Convertiamo i risultati in un DataFrame per una visualizzazione più chiara
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Ordina i risultati in base all'OSS in ordine decrescente
df_top_oss = df_top_oss.sort_values(by='OSS', ascending=False)

# Mostra i risultati solo per quelli che superano la soglia
df_top_oss  # Mostra i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
147,note,provider_id,patients,birthdate,0.666667
153,note,visit_occurrence_id,patients,birthdate,0.666667
117,note,note_date,patients,birthdate,0.666667
150,note,visit_detail_id,patients,birthdate,0.666667
149,note,provider_id,immunizations,date,0.500000
...,...,...,...,...,...
18,cohort_definition,cohort_definition_name,patients,gender,0.100000
177,payer_plan_period,person_id,patients,ssn,0.100000
178,payer_plan_period,plan_source_value,encounters,reasoncode,0.100000
214,person,provider_id,allergies,description,0.100000


In [ ]:
from collections import defaultdict

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo)
pretty_name_by_attribute = AnnotazioneMedCAT.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute.get(attribute1, set())
    P2 = pretty_name_by_attribute.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))
    union = len(P1.union(P2))

    if union == 0:  # Evitiamo la divisione per zero
        return 0.0
    return intersection / union

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA)
top_oss_results = defaultdict(list)

# Calcolare OSS tra tutte le combinazioni di attributi
for (tableA, attributeA) in pretty_name_by_attribute.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute.keys():
        # Assicuriamoci che la TabellaA e TabellaB siano diverse
        if tableA != tableB:
            oss = calculate_oss((tableA, attributeA), (tableB, attributeB))
            oss_list.append(((tableB, attributeB), oss))

    # Ordiniamo per similarità OSS in ordine decrescente e prendiamo i primi 3
    top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]

# Convertiamo i risultati in DataFrame per una visualizzazione più chiara
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Mostriamo i risultati
df_top_oss.head(10)  # Visualizza i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
0,admissions,admission_location,admissions_hospital,expire_flag,0.454545
1,admissions,admission_location,transfers,mimic_id,0.400000
2,admissions,admission_location,transfers,prev_wardid,0.333333
3,admissions,admission_type,admissions_hospital,expire_flag,0.625000
4,admissions,admission_type,services,mimic_id,0.428571
5,admissions,admission_type,inputevents_cv,mimic_id,0.400000
6,admissions,hadm_id,admissions_hospital,expire_flag,0.625000
7,admissions,hadm_id,services,mimic_id,0.428571
8,admissions,hadm_id,inputevents_cv,mimic_id,0.400000
9,admissions,marital_status,admissions_hospital,expire_flag,0.555556


##DF CON CONTEXT SIMILARITY = 1

In [ ]:
AnnotazioneMedCAT_MAX.columns= ['attribute', 'detected_name', 'pretty_name', 'context_similarity']

In [ ]:
# aggiungo anche nome tabella e nome attributo
AnnotazioneMedCAT_MAX = AnnotazioneMedCAT_MAX.copy()

# Separiamo la colonna 'attribute' in base all'ultimo '_', separando in 3 parti
split_columns = AnnotazioneMedCAT_MAX['attribute'].str.rsplit('_', n=2, expand=True)

# Usa .loc per assegnare le nuove colonne al DataFrame originale
AnnotazioneMedCAT_MAX.loc[:, 'Tabella'] = split_columns[0]
AnnotazioneMedCAT_MAX.loc[:, 'Attributo'] = split_columns[1] + '_' + split_columns[2]

# Visualizza il risultato
AnnotazioneMedCAT_MAX

,attribute,detected_name,pretty_name,context_similarity,Tabella,Attributo
0,beneficiarysummary_bene_birth_dt,date~of~birth,Date of birth,1.0,beneficiarysummary_bene,birth_dt
1,beneficiarysummary_bene_death_dt,date~of~death,Date of death,1.0,beneficiarysummary_bene,death_dt
2,beneficiarysummary_bene_esrd_ind,end~stage~renal~disease,End-stage renal disease,1.0,beneficiarysummary_bene,esrd_ind
3,beneficiarysummary_sp_state_code,state,State,1.0,beneficiarysummary_sp,state_code
5,beneficiarysummary_bene_hi_cvrage_tot_mons,of~part,In (attribute),1.0,beneficiarysummary_bene_hi_cvrage,tot_mons
...,...,...,...,...,...,...
5912,providers_state,abbreviation,Abbreviation,1.0,providers,NaN
5913,providers_zip,patient,Patient,1.0,providers,NaN
5914,providers_zip,street~address,Street address,1.0,providers,NaN
5915,providers_utilization,patient,Patient,1.0,providers,NaN


il dataframe GMT ha come columns ['SOURCE', 'GAT', 'LAT']
dove GAT corrisponde ad attribute AnnotazioneMedCAT con l'unica differenza che mentre in GMT
ho un GAT ad esempio beneficiarysummary_bene_birth_dt
in AnnotazioneMedCAT me lo trovo riscritto come beneficiarysummary-bene_birth_dt
cioè il nome della tabella è separato da _ nel primo e da - nel secondo:
quindi prima di fare join devo uniformare

La GMT mantiene le corrispondenze di ogni singolo schema delle tre sorgenti rispetto allo schema OMOP
(siamo nella situazione top-down)


In [ ]:
TuttiGliSchemi.columns

Index(['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo'], dtype='object')

In [ ]:
# Eseguiamo il merge con AnnotazioneMedCAT
merged_df = TuttiGliSchemi.merge(AnnotazioneMedCAT_MAX, left_on='AttributoNomeCompleto', right_on='attribute', how='inner')

# Visualizziamo i primi risultati
merged_df.head()

NewAnnotazioneMedCAT=merged_df[['SOURCE', 'AttributoNomeCompleto', 'Tabella_x', 'Attributo_x', 'detected_name','pretty_name', 'context_similarity' ]]

In [ ]:
NewAnnotazioneMedCAT.head()
AnnotazioneMedCAT_MAX[AnnotazioneMedCAT_MAX['attribute'].isin(NewAnnotazioneMedCAT['AttributoNomeCompleto'])]

,attribute,detected_name,pretty_name,context_similarity,Tabella,Attributo
0,beneficiarysummary_bene_birth_dt,date~of~birth,Date of birth,1.0,beneficiarysummary_bene,birth_dt
1,beneficiarysummary_bene_death_dt,date~of~death,Date of death,1.0,beneficiarysummary_bene,death_dt
2,beneficiarysummary_bene_esrd_ind,end~stage~renal~disease,End-stage renal disease,1.0,beneficiarysummary_bene,esrd_ind
3,beneficiarysummary_sp_state_code,state,State,1.0,beneficiarysummary_sp,state_code
5,beneficiarysummary_bene_hi_cvrage_tot_mons,of~part,In (attribute),1.0,beneficiarysummary_bene_hi_cvrage,tot_mons
...,...,...,...,...,...,...
5912,providers_state,abbreviation,Abbreviation,1.0,providers,NaN
5913,providers_zip,patient,Patient,1.0,providers,NaN
5914,providers_zip,street~address,Street address,1.0,providers,NaN
5915,providers_utilization,patient,Patient,1.0,providers,NaN


In [ ]:
NewAnnotazioneMedCAT[['SOURCE']].drop_duplicates()

,SOURCE
0,Synthea
331,Csm
525,Mimic
1556,OMOP


In [ ]:
NewAnnotazioneMedCAT.columns=['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo',
       'detected_name', 'pretty_name', 'context_similarity']

In [ ]:
# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Mimic']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

In [ ]:
omop_data

,SOURCE,AttributoNomeCompleto,Tabella,Attributo,detected_name,pretty_name,context_similarity
1556,OMOP,person_person_id,person,person_id,person,Person,1.0
1557,OMOP,person_person_id,person,person_id,domain,Concept model domain,1.0
1558,OMOP,person_person_id,person,person_id,contain,Containing,1.0
1559,OMOP,person_person_id,person,person_id,identify,Identified,1.0
1560,OMOP,person_person_id,person,person_id,patient,Patient,1.0
...,...,...,...,...,...,...,...
4677,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,physician,Physician,1.0
4678,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,nurse,Professional nurse,1.0
4679,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,midwife,Registered midwife,1.0
4680,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,physical~therapist,Physiotherapist,1.0


In [ ]:
mimic_data

,SOURCE,AttributoNomeCompleto,Tabella,Attributo,detected_name,pretty_name,context_similarity
525,Mimic,admissions_mimic_id,admissions,mimic_id,admission,Hospital admission,1.0
526,Mimic,admissions_mimic_id,admissions,mimic_id,table,"Table, device",1.0
527,Mimic,admissions_mimic_id,admissions,mimic_id,give~information,Informing,1.0
528,Mimic,admissions_mimic_id,admissions,mimic_id,patient,Patient,1.0
529,Mimic,admissions_mimic_id,admissions,mimic_id,admission,Hospital admission,1.0
...,...,...,...,...,...,...,...
1551,Mimic,transfers_mimic_id,transfers,mimic_id,physical,Physical,1.0
1552,Mimic,transfers_mimic_id,transfers,mimic_id,location,Location,1.0
1553,Mimic,transfers_mimic_id,transfers,mimic_id,patient,Patient,1.0
1554,Mimic,transfers_mimic_id,transfers,mimic_id,hospital,Hospital,1.0


In [ ]:
from collections import defaultdict
import pandas as pd

# Definisci una soglia di similarità per OSS
oss_threshold = 0.5  # Puoi modificarla in base alle tue necessità

# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Mimic']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute_omop.get(attribute1, set())
    P2 = pretty_name_by_attribute_mimic.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))
    union = len(P1.union(P2))

    if union == 0:  # Evitiamo la divisione per zero
        return 0.0
    return intersection / union

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA) di OMOP
top_oss_results = defaultdict(list)
no_match_results = []  # Lista per gli attributi che non hanno corrispondenza

# Calcolare OSS tra tutte le combinazioni di attributi di OMOP e MIMIC
for (tableA, attributeA) in pretty_name_by_attribute_omop.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute_mimic.keys():
        oss = calculate_oss((tableA, attributeA), (tableB, attributeB))

        if oss >= oss_threshold:
            oss_list.append(((tableB, attributeB), oss))

    # Se ci sono corrispondenze sopra la soglia, prendi le prime 3
    if oss_list:
        top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]
    else:
        # Se non ci sono corrispondenze sopra la soglia, aggiungi "assenza"
        no_match_results.append((tableA, attributeA, "Assente in MIMIC", "Assente in MIMIC", 0.0))

# Aggiungi i risultati di "assenza" al dizionario finale
df_no_match = pd.DataFrame(no_match_results, columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Convertiamo i risultati delle corrispondenze in un DataFrame
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Unisci i risultati di corrispondenza con quelli di "assenza"
df_final_result = pd.concat([df_top_oss, df_no_match], ignore_index=True)

# Ordina i risultati in base all'OSS in ordine decrescente
df_final_result = df_final_result.sort_values(by='OSS', ascending=False)

# Visualizza i risultati
df_final_result.head(10)  # Mostra i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
45,note,note_date,outputevents,mimic_id,1.0
43,note,note_date,inputevents_cv,mimic_id,1.0
73,note,visit_detail_id,inputevents_cv,mimic_id,1.0
64,note,note_title,inputevents_cv,mimic_id,1.0
75,note,visit_detail_id,outputevents,mimic_id,1.0
65,note,note_title,inputevents_mv,mimic_id,1.0
72,note,provider_id,outputevents,mimic_id,1.0
71,note,provider_id,inputevents_mv,mimic_id,1.0
76,note,visit_occurrence_id,inputevents_cv,mimic_id,1.0
77,note,visit_occurrence_id,inputevents_mv,mimic_id,1.0


In [ ]:
from collections import defaultdict
import pandas as pd

# Definisci una soglia di similarità per OSS
oss_threshold = 0.1  # Puoi modificarla in base alle tue necessità

# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Synthea']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute_omop.get(attribute1, set())
    P2 = pretty_name_by_attribute_mimic.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))
    union = len(P1.union(P2))

    if union == 0:  # Evitiamo la divisione per zero
        return 0.0
    return intersection / union

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA) di OMOP
top_oss_results = defaultdict(list)

# Calcolare OSS tra tutte le combinazioni di attributi di OMOP e MIMIC
for (tableA, attributeA) in pretty_name_by_attribute_omop.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute_mimic.keys():
        oss = calculate_oss((tableA, attributeA), (tableB, attributeB))

        if oss >= oss_threshold:
            oss_list.append(((tableB, attributeB), oss))

    # Se ci sono corrispondenze sopra la soglia, prendi le prime 3
    if oss_list:
        top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]

# Convertiamo i risultati in un DataFrame per una visualizzazione più chiara
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Ordina i risultati in base all'OSS in ordine decrescente
df_top_oss = df_top_oss.sort_values(by='OSS', ascending=False)

# Mostra i risultati solo per quelli che superano la soglia
df_top_oss  # Mostra i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
138,note,note_id,patients,id,0.666667
139,note,note_id,patients,passport,0.666667
128,note,note_datetime,encounters,stop,0.666667
137,note,note_id,encounters,id,0.666667
155,note,provider_id,allergies,code,0.500000
...,...,...,...,...,...
97,drug_exposure,drug_exposure_id,careplans,id,0.100000
96,death,person_id,patients,passport,0.100000
95,death,person_id,patients,id,0.100000
94,death,person_id,encounters,id,0.100000


In [ ]:
from collections import defaultdict

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo)
pretty_name_by_attribute = AnnotazioneMedCAT_MAX.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute.get(attribute1, set())
    P2 = pretty_name_by_attribute.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))
    union = len(P1.union(P2))

    if union == 0:  # Evitiamo la divisione per zero
        return 0.0
    return intersection / union

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA)
top_oss_results = defaultdict(list)

# Calcolare OSS tra tutte le combinazioni di attributi
for (tableA, attributeA) in pretty_name_by_attribute.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute.keys():
        # Assicuriamoci che la TabellaA e TabellaB siano diverse
        if tableA != tableB:
            oss = calculate_oss((tableA, attributeA), (tableB, attributeB))
            oss_list.append(((tableB, attributeB), oss))

    # Ordiniamo per similarità OSS in ordine decrescente e prendiamo i primi 3
    top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]

# Convertiamo i risultati in DataFrame per una visualizzazione più chiara
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Mostriamo i risultati
df_top_oss.head(10)  # Visualizza i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
0,admissions,admission_location,admissions_hospital,expire_flag,0.555556
1,admissions,admission_location,transfers,mimic_id,0.400000
2,admissions,admission_location,transfers,prev_wardid,0.363636
3,admissions,admission_type,admissions_hospital,expire_flag,0.833333
4,admissions,admission_type,services,mimic_id,0.428571
5,admissions,admission_type,inputevents_cv,mimic_id,0.400000
6,admissions,hadm_id,admissions_hospital,expire_flag,0.833333
7,admissions,hadm_id,services,mimic_id,0.428571
8,admissions,hadm_id,inputevents_cv,mimic_id,0.400000
9,admissions,marital_status,admissions_hospital,expire_flag,0.714286


##DF CON CONTEXT SIMILARITY = 0.8

In [ ]:
AnnotazioneMedCAT_08.columns= ['attribute', 'detected_name', 'pretty_name', 'context_similarity']

In [ ]:
# aggiungo anche nome tabella e nome attributo
AnnotazioneMedCAT_08 = AnnotazioneMedCAT_08.copy()

# Separiamo la colonna 'attribute' in base all'ultimo '_', separando in 3 parti
split_columns = AnnotazioneMedCAT_08['attribute'].str.rsplit('_', n=2, expand=True)

# Usa .loc per assegnare le nuove colonne al DataFrame originale
AnnotazioneMedCAT_08.loc[:, 'Tabella'] = split_columns[0]
AnnotazioneMedCAT_08.loc[:, 'Attributo'] = split_columns[1] + '_' + split_columns[2]

# Visualizza il risultato
AnnotazioneMedCAT_08

,attribute,detected_name,pretty_name,context_similarity,Tabella,Attributo
0,beneficiarysummary_bene_birth_dt,date~of~birth,Date of birth,1.00,beneficiarysummary_bene,birth_dt
1,beneficiarysummary_bene_death_dt,date~of~death,Date of death,1.00,beneficiarysummary_bene,death_dt
2,beneficiarysummary_bene_esrd_ind,end~stage~renal~disease,End-stage renal disease,1.00,beneficiarysummary_bene,esrd_ind
3,beneficiarysummary_sp_state_code,state,State,1.00,beneficiarysummary_sp,state_code
4,beneficiarysummary_bene_hi_cvrage_tot_mons,total,Total,0.99,beneficiarysummary_bene_hi_cvrage,tot_mons
...,...,...,...,...,...,...
5912,providers_state,abbreviation,Abbreviation,1.00,providers,NaN
5913,providers_zip,patient,Patient,1.00,providers,NaN
5914,providers_zip,street~address,Street address,1.00,providers,NaN
5915,providers_utilization,patient,Patient,1.00,providers,NaN


il dataframe GMT ha come columns ['SOURCE', 'GAT', 'LAT']
dove GAT corrisponde ad attribute AnnotazioneMedCAT con l'unica differenza che mentre in GMT
ho un GAT ad esempio beneficiarysummary_bene_birth_dt
in AnnotazioneMedCAT me lo trovo riscritto come beneficiarysummary-bene_birth_dt
cioè il nome della tabella è separato da _ nel primo e da - nel secondo:
quindi prima di fare join devo uniformare

La GMT mantiene le corrispondenze di ogni singolo schema delle tre sorgenti rispetto allo schema OMOP
(siamo nella situazione top-down)


In [ ]:
TuttiGliSchemi.columns

Index(['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo'], dtype='object')

In [ ]:
# Eseguiamo il merge con AnnotazioneMedCAT
merged_df = TuttiGliSchemi.merge(AnnotazioneMedCAT_08, left_on='AttributoNomeCompleto', right_on='attribute', how='inner')

# Visualizziamo i primi risultati
merged_df.head()

NewAnnotazioneMedCAT=merged_df[['SOURCE', 'AttributoNomeCompleto', 'Tabella_x', 'Attributo_x', 'detected_name','pretty_name', 'context_similarity' ]]

In [ ]:
NewAnnotazioneMedCAT.head()
AnnotazioneMedCAT_08[AnnotazioneMedCAT_08['attribute'].isin(NewAnnotazioneMedCAT['AttributoNomeCompleto'])]

,attribute,detected_name,pretty_name,context_similarity,Tabella,Attributo
0,beneficiarysummary_bene_birth_dt,date~of~birth,Date of birth,1.00,beneficiarysummary_bene,birth_dt
1,beneficiarysummary_bene_death_dt,date~of~death,Date of death,1.00,beneficiarysummary_bene,death_dt
2,beneficiarysummary_bene_esrd_ind,end~stage~renal~disease,End-stage renal disease,1.00,beneficiarysummary_bene,esrd_ind
3,beneficiarysummary_sp_state_code,state,State,1.00,beneficiarysummary_sp,state_code
4,beneficiarysummary_bene_hi_cvrage_tot_mons,total,Total,0.99,beneficiarysummary_bene_hi_cvrage,tot_mons
...,...,...,...,...,...,...
5912,providers_state,abbreviation,Abbreviation,1.00,providers,NaN
5913,providers_zip,patient,Patient,1.00,providers,NaN
5914,providers_zip,street~address,Street address,1.00,providers,NaN
5915,providers_utilization,patient,Patient,1.00,providers,NaN


In [ ]:
NewAnnotazioneMedCAT[['SOURCE']].drop_duplicates()

,SOURCE
0,Synthea
358,Csm
560,Mimic
1647,OMOP


In [ ]:
NewAnnotazioneMedCAT.columns=['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo',
       'detected_name', 'pretty_name', 'context_similarity']

In [ ]:
# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Mimic']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

In [ ]:
omop_data

,SOURCE,AttributoNomeCompleto,Tabella,Attributo,detected_name,pretty_name,context_similarity
1647,OMOP,person_person_id,person,person_id,person,Person,1.00
1648,OMOP,person_person_id,person,person_id,domain,Concept model domain,1.00
1649,OMOP,person_person_id,person,person_id,contain,Containing,1.00
1650,OMOP,person_person_id,person,person_id,identify,Identified,1.00
1651,OMOP,person_person_id,person,person_id,patient,Patient,1.00
...,...,...,...,...,...,...,...
5042,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,nurse,Professional nurse,1.00
5043,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,midwife,Registered midwife,1.00
5044,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,physical~therapist,Physiotherapist,1.00
5045,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,refer,Patient referral,1.00


In [ ]:
mimic_data

,SOURCE,AttributoNomeCompleto,Tabella,Attributo,detected_name,pretty_name,context_similarity
560,Mimic,admissions_mimic_id,admissions,mimic_id,admission,Hospital admission,1.0
561,Mimic,admissions_mimic_id,admissions,mimic_id,table,"Table, device",1.0
562,Mimic,admissions_mimic_id,admissions,mimic_id,give~information,Informing,1.0
563,Mimic,admissions_mimic_id,admissions,mimic_id,patient,Patient,1.0
564,Mimic,admissions_mimic_id,admissions,mimic_id,admission,Hospital admission,1.0
...,...,...,...,...,...,...,...
1642,Mimic,transfers_mimic_id,transfers,mimic_id,physical,Physical,1.0
1643,Mimic,transfers_mimic_id,transfers,mimic_id,location,Location,1.0
1644,Mimic,transfers_mimic_id,transfers,mimic_id,patient,Patient,1.0
1645,Mimic,transfers_mimic_id,transfers,mimic_id,hospital,Hospital,1.0


In [ ]:
from collections import defaultdict
import pandas as pd

# Definisci una soglia di similarità per OSS
oss_threshold = 0.5  # Puoi modificarla in base alle tue necessità

# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Mimic']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute_omop.get(attribute1, set())
    P2 = pretty_name_by_attribute_mimic.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))
    union = len(P1.union(P2))

    if union == 0:  # Evitiamo la divisione per zero
        return 0.0
    return intersection / union

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA) di OMOP
top_oss_results = defaultdict(list)
no_match_results = []  # Lista per gli attributi che non hanno corrispondenza

# Calcolare OSS tra tutte le combinazioni di attributi di OMOP e MIMIC
for (tableA, attributeA) in pretty_name_by_attribute_omop.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute_mimic.keys():
        oss = calculate_oss((tableA, attributeA), (tableB, attributeB))

        if oss >= oss_threshold:
            oss_list.append(((tableB, attributeB), oss))

    # Se ci sono corrispondenze sopra la soglia, prendi le prime 3
    if oss_list:
        top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]
    else:
        # Se non ci sono corrispondenze sopra la soglia, aggiungi "assenza"
        no_match_results.append((tableA, attributeA, "Assente in MIMIC", "Assente in MIMIC", 0.0))

# Aggiungi i risultati di "assenza" al dizionario finale
df_no_match = pd.DataFrame(no_match_results, columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Convertiamo i risultati delle corrispondenze in un DataFrame
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Unisci i risultati di corrispondenza con quelli di "assenza"
df_final_result = pd.concat([df_top_oss, df_no_match], ignore_index=True)

# Ordina i risultati in base all'OSS in ordine decrescente
df_final_result = df_final_result.sort_values(by='OSS', ascending=False)

# Visualizza i risultati
df_final_result.head(10)  # Mostra i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
65,note,visit_detail_id,inputevents_cv,mimic_id,1.0
70,note,visit_occurrence_id,outputevents,mimic_id,1.0
69,note,visit_occurrence_id,inputevents_mv,mimic_id,1.0
68,note,visit_occurrence_id,inputevents_cv,mimic_id,1.0
67,note,visit_detail_id,outputevents,mimic_id,1.0
37,note,note_date,outputevents,mimic_id,1.0
36,note,note_date,inputevents_mv,mimic_id,1.0
66,note,visit_detail_id,inputevents_mv,mimic_id,1.0
35,note,note_date,inputevents_cv,mimic_id,1.0
64,note,provider_id,outputevents,mimic_id,1.0


In [ ]:
from collections import defaultdict
import pandas as pd

# Definisci una soglia di similarità per OSS
oss_threshold = 0.1  # Puoi modificarla in base alle tue necessità

# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Synthea']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute_omop.get(attribute1, set())
    P2 = pretty_name_by_attribute_mimic.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))
    union = len(P1.union(P2))

    if union == 0:  # Evitiamo la divisione per zero
        return 0.0
    return intersection / union

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA) di OMOP
top_oss_results = defaultdict(list)

# Calcolare OSS tra tutte le combinazioni di attributi di OMOP e MIMIC
for (tableA, attributeA) in pretty_name_by_attribute_omop.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute_mimic.keys():
        oss = calculate_oss((tableA, attributeA), (tableB, attributeB))

        if oss >= oss_threshold:
            oss_list.append(((tableB, attributeB), oss))

    # Se ci sono corrispondenze sopra la soglia, prendi le prime 3
    if oss_list:
        top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]

# Convertiamo i risultati in un DataFrame per una visualizzazione più chiara
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Ordina i risultati in base all'OSS in ordine decrescente
df_top_oss = df_top_oss.sort_values(by='OSS', ascending=False)

# Mostra i risultati solo per quelli che superano la soglia
df_top_oss  # Mostra i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
133,note,note_id,patients,passport,0.666667
132,note,note_id,patients,id,0.666667
131,note,note_id,encounters,id,0.666667
122,note,note_datetime,encounters,stop,0.666667
130,note,note_event_id,patients,passport,0.500000
...,...,...,...,...,...
183,payer_plan_period,person_id,patients,id,0.100000
182,payer_plan_period,person_id,encounters,id,0.100000
181,payer_plan_period,payer_source_value,encounters,reasoncode,0.100000
303,specimen,disease_status_concept_id,patients,id,0.100000


In [ ]:
from collections import defaultdict

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo)
pretty_name_by_attribute = AnnotazioneMedCAT_08.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute.get(attribute1, set())
    P2 = pretty_name_by_attribute.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))
    union = len(P1.union(P2))

    if union == 0:  # Evitiamo la divisione per zero
        return 0.0
    return intersection / union

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA)
top_oss_results = defaultdict(list)

# Calcolare OSS tra tutte le combinazioni di attributi
for (tableA, attributeA) in pretty_name_by_attribute.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute.keys():
        # Assicuriamoci che la TabellaA e TabellaB siano diverse
        if tableA != tableB:
            oss = calculate_oss((tableA, attributeA), (tableB, attributeB))
            oss_list.append(((tableB, attributeB), oss))

    # Ordiniamo per similarità OSS in ordine decrescente e prendiamo i primi 3
    top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]

# Convertiamo i risultati in DataFrame per una visualizzazione più chiara
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Mostriamo i risultati
df_top_oss.head(10)  # Visualizza i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
0,admissions,admission_location,admissions_hospital,expire_flag,0.555556
1,admissions,admission_location,transfers,mimic_id,0.400000
2,admissions,admission_location,transfers,prev_wardid,0.363636
3,admissions,admission_type,admissions_hospital,expire_flag,0.833333
4,admissions,admission_type,services,mimic_id,0.428571
5,admissions,admission_type,inputevents_cv,mimic_id,0.400000
6,admissions,hadm_id,admissions_hospital,expire_flag,0.833333
7,admissions,hadm_id,services,mimic_id,0.428571
8,admissions,hadm_id,inputevents_cv,mimic_id,0.400000
9,admissions,marital_status,admissions_hospital,expire_flag,0.714286


# 3 FEBBRAIO - Annotazione rispetto a SNOMED

Domenico 3 Febbraio

Carico il file dataframe_snomed.csv ricevuto da Gabriele, per un'analisi e confronto veloce

In [ ]:
#medcat full
AnnotazioneMedCAT

#medcat cs = 1
AnnotazioneMedCAT_MAX

#medcat cs >= .8
AnnotazioneMedCAT_08

,text,entity_id,pretty_name,cui,type_ids,types,source_value,detected_name,acc,context_similarity,ontologies,names
0,"beneficiarysummary pertains to a synthetic medicare beneficiary , date of birth",1,Date of birth,184099003,['2680757'],['observable entity'],date of birth,date~of~birth,1.00,1.00,['SNOMED-CT'],beneficiarysummary_bene_birth_dt
1,"beneficiarysummary pertains to a synthetic medicare beneficiary , date of death",1,Date of death,399753006,['2680757'],['observable entity'],date of death,date~of~death,1.00,1.00,['SNOMED-CT'],beneficiarysummary_bene_death_dt
2,"beneficiarysummary pertains to a synthetic medicare beneficiary , end stage renal disease indicator",2,End-stage renal disease,46177005,['9090192'],['disorder'],end stage renal disease,end~stage~renal~disease,1.00,1.00,['SNOMED-CT'],beneficiarysummary_bene_esrd_ind
3,"beneficiarysummary pertains to a synthetic medicare beneficiary , state code",0,State,398070004,['75168589'],['environment'],state,state,1.00,1.00,['SNOMED-CT'],beneficiarysummary_sp_state_code
4,"beneficiarysummary pertains to a synthetic medicare beneficiary , total number of months of part a coverage for the beneficiary.",0,Total,255619001,['7882689'],['qualifier value'],total,total,0.99,0.99,['SNOMED-CT'],beneficiarysummary_bene_hi_cvrage_tot_mons
...,...,...,...,...,...,...,...,...,...,...,...,...
5912,"clinicians that provide patient care., street address state abbreviation.",5,Abbreviation,398223008,['7882689'],['qualifier value'],abbreviation,abbreviation,1.00,1.00,['SNOMED-CT'],providers_state
5913,"clinicians that provide patient care., street address zip or postal code.",0,Patient,116154003,['31601201'],['person'],patient,patient,1.00,1.00,['SNOMED-CT'],providers_zip
5914,"clinicians that provide patient care., street address zip or postal code.",2,Street address,398099009,['2680757'],['observable entity'],street address,street~address,1.00,1.00,['SNOMED-CT'],providers_zip
5915,"clinicians that provide patient care., the number of encounter's performed by this provider.",0,Patient,116154003,['31601201'],['person'],patient,patient,1.00,1.00,['SNOMED-CT'],providers_utilization


In [ ]:
# se considero tutte le sorgenti e OMOP_CDM
TuttiGliSchemi

,SOURCE,AttributoNomeCompleto,Tabella,Attributo
0,Synthea,allergies_start,allergies,start
1,Synthea,allergies_stop,allergies,stop
2,Synthea,allergies_patient,allergies,patient
3,Synthea,allergies_encounter,allergies,encounter
4,Synthea,allergies_code,allergies,code
...,...,...,...,...
29082,OMOP,provider_provider_source_value,provider,provider_source_value
29193,OMOP,provider_specialty_source_value,provider,specialty_source_value
29304,OMOP,provider_specialty_source_concept_id,provider,specialty_source_concept_id
29415,OMOP,provider_gender_source_value,provider,gender_source_value


In [ ]:
merged_df.loc[merged_df['_merge'] == 'right_only', 'AttributoNomeCompleto']

156       beneficiarysummary_bene_county_cd
162         beneficiarysummary_bene_race_cd
163    beneficiarysummary_bene_sex_ident_cd
172          beneficiarysummary_desynpuf_id
Name: AttributoNomeCompleto, dtype: object

In [ ]:
merged_df.columns

Index(['text', 'entity_id', 'pretty_name', 'cui', 'type_ids', 'types',
       'source_value', 'detected_name', 'acc', 'context_similarity',
       'ontologies', 'names', 'SOURCE', 'AttributoNomeCompleto', 'Tabella',
       'Attributo', '_merge'],
      dtype='object')

In [ ]:
# on AttributoNomeCompleto
# medcat df full
Medcat_Df_Full_AttributoNomeCompleto = AnnotazioneMedCAT.merge(TuttiGliSchemi, left_on='names', right_on='AttributoNomeCompleto', how='outer', indicator=True)

# medcat cs=1
Medcat_Df_CS_1_AttributoNomeCompleto = AnnotazioneMedCAT_MAX.merge(TuttiGliSchemi, left_on='names', right_on='AttributoNomeCompleto', how='outer', indicator=True)

# medcat cs>=0.8
Medcat_Df_CS_08_AttributoNomeCompleto= AnnotazioneMedCAT_08.merge(TuttiGliSchemi, left_on='names', right_on='AttributoNomeCompleto', how='outer', indicator=True)



In [ ]:
Medcat_Df_Full_AttributoNomeCompleto


,text,entity_id,pretty_name,cui,type_ids,types,source_value,detected_name,acc,context_similarity,ontologies,names,SOURCE,AttributoNomeCompleto,Tabella,Attributo,_merge
0,"the admissions table gives information regarding a patientâs admission to the hospital., admission_locationÂ provides information about the previous location of the patient prior to arriving at the hospital.",0.0,Hospital admission,32485007.0,['28321150'],['procedure'],admissions,admission,1.000000,1.000000,['SNOMED-CT'],admissions_admission_location,Mimic,admissions_admission_location,admissions,admission_location,both
1,"the admissions table gives information regarding a patientâs admission to the hospital., admission_locationÂ provides information about the previous location of the patient prior to arriving at the hospital.",1.0,"Table, device",86407004.0,['32816260'],['physical object'],table,table,1.000000,1.000000,['SNOMED-CT'],admissions_admission_location,Mimic,admissions_admission_location,admissions,admission_location,both
2,"the admissions table gives information regarding a patientâs admission to the hospital., admission_locationÂ provides information about the previous location of the patient prior to arriving at the hospital.",3.0,Informing,223458004.0,['28321150'],['procedure'],gives information,give~information,1.000000,1.000000,['SNOMED-CT'],admissions_admission_location,Mimic,admissions_admission_location,admissions,admission_location,both
3,"the admissions table gives information regarding a patientâs admission to the hospital., admission_locationÂ provides information about the previous location of the patient prior to arriving at the hospital.",4.0,Patient,116154003.0,['31601201'],['person'],patient,patient,1.000000,1.000000,['SNOMED-CT'],admissions_admission_location,Mimic,admissions_admission_location,admissions,admission_location,both
4,"the admissions table gives information regarding a patientâs admission to the hospital., admission_locationÂ provides information about the previous location of the patient prior to arriving at the hospital.",5.0,Hospital admission,32485007.0,['28321150'],['procedure'],admission,admission,1.000000,1.000000,['SNOMED-CT'],admissions_admission_location,Mimic,admissions_admission_location,admissions,admission_location,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5925,"the visit_occurrence table contains the spans of time a person continuously receives medical services from one or more providers at a care site in a given setting within the health care system. visits are classified into 4 settings: outpatient care, inpatient confinement, emergency room, and long-term care. persons may transition between these settings over the course of an episode of care (for example, treatment of a disease onset)., a foreign key to the predefined concept identifier in the standardized vocabularies reflecting the type of source data from which the visit record is derived belonging to the 'visit type' vocabulary.",21.0,Episode,263503000.0,['43039974'],['attribute'],episode of,episode~of,1.000000,1.000000,['SNOMED-CT'],visit_occurrence_visit_type_concept_id,OMOP,visit_occurrence_visit_type_concept_id,visit_occurrence,visit_type_concept_id,both
5926,"the visit_occurrence table contains the spans of time a person continuously receives medical services from one or more providers at a care site in a given setting within the health care system. visits are classified into 4 settings: outpatient care, inpatient confinement, emergency room, and long-term care. persons may transition between these settings over the course of an episode of care (for example, treatment of a disease onset)., a foreign key to the predefined concept identifier in the standardized vocabularies reflecting the type of source data from which the visit record is derived belonging to the 'visit type' vocabulary.",24.0,Onset of illness,217315002.0,['67667581'],['finding'],disease onset,disease~onset,1.000000,1.000000,['SN

In [ ]:
# on Attributo
# medcat df full
Medcat_Df_Full_Attributo = AnnotazioneMedCAT.merge(TuttiGliSchemi, left_on='names', right_on='Attributo', how='outer', indicator=True)

# medcat cs=1
Medcat_Df_CS_1_Attributo = AnnotazioneMedCAT_MAX.merge(TuttiGliSchemi, left_on='names', right_on='Attributo', how='outer', indicator=True)

# medcat cs>=0.8
Medcat_Df_CS_08_Attributo= AnnotazioneMedCAT_08.merge(TuttiGliSchemi, left_on='names', right_on='Attributo', how='outer', indicator=True)



In [ ]:
# on Tabella
# medcat df full
Medcat_Df_Full_Tabella = AnnotazioneMedCAT.merge(TuttiGliSchemi, left_on='names', right_on='Tabella', how='outer', indicator=True)

# medcat cs=1
Medcat_Df_CS_1_Tabella = AnnotazioneMedCAT_MAX.merge(TuttiGliSchemi, left_on='names', right_on='Tabella', how='outer', indicator=True)

# medcat cs>=0.8
Medcat_Df_CS_08_Tabella= AnnotazioneMedCAT_08.merge(TuttiGliSchemi, left_on='names', right_on='Tabella', how='outer', indicator=True)



In [ ]:
# aggiungere una sigla che sintetizza l'annotazione



### POI FAREMO QUELLE PER

# BioPortal_SoloDescrizioneAttributo_SNOMEDCT
# per dire che l'annotazione è stata fatta con BioPortal sulla descrizione del solo attributo  rispetto a SNOMEDCT








## Creazione dataframe unico

La prima cosa da fare è di consolidare in un unico file (lo chiameremo Schemi e lo salveremo su github ) gli schemi di tutte le 4 sorgenti

Questo dataframe Schemi avrà le seguenti colonne:

- SOURCE
- Tabella
- Attributo
- AttributoNomeCompleto  (che  coincide con names , controllare e verificare anche che questo coincide con  Tabella _ Attributo

Nota: mi sembra che AttributoNomeCompleto non sia univoco per tutte le sorgenti, se ricordo bene ce ne erano solo due di attributi con lo stesso nome- ricordiamoci questo aspetto e quindi di usare sempre anche SOURCE )

DescrizioneCompleta (che coincide con quella che si chiama ora names)
DescrizioneTabella
DescrizioneAttributo

### Direi di fare un unico dataframe ANNOTAZIONI con tutti i campi necessari, che caso per caso possono assumere valori nulli se non attinenti al metodo di annotazione

Questa una prima proposta Sempre con riferimento a

     Medcat_Df_Full_AttributoNomeCompleto



Gli togliamo tutti i campi messi in Schemi, lasciandovi ovviamente una foreign key

SOURCE

AttributoNomeCompleto

Poi aggiungiamo un campo

TipoDiAnnotazione

Che assume un valore descrittivo tipo ‘Medcat_ Full_AttributoNomeCompleto’

Valore che andremo opportunamente a codificare

Quindi gli altri campi attuali

ontologies : questa la possiamo chiamare Ontology in quanto al momento è una sola
entity_id (non ricordo più il significato di questo campo? È univoco progressivo per tutto il dataframe? Sarebbe meglio averlo univoco progressivo per un attributo, per un TipoDiAnnotazione e per una Ontology)
pretty_name
cui
type_ids
types
source_value
detected_name
acc
context_similarity

# Overlap e Jaccard

quella considerata sopra non è l'overlap ma la jaccard

la overlap corretta è senza il divisore

### Overlap Set Similarity (OSS)

La **Overlap Set Similarity** tra due *text* (ad esempio, *att1* e *att2*) è definita come la misura del grado di sovrapposizione tra i rispettivi insiemi di *pretty_name*. Formalmente:

$$
\text{OSS}(att1, att2) = |P_{att1} \cap P_{att2}|
$$

dove:

- \(P_{att1}\) e \(P_{att2}\) sono gli insiemi di *pretty_name* associati a *att1* e *att2*, rispettivamente.
- \(|P_{att1} \cap P_{att2}|\) rappresenta il numero di *pretty_name* in comune tra i due attributi.


SI USA GENERALMENET LA JACCARD, MA DEVO TROVARE ARTICOLO IN CUI SI DICE PERCHE' USARE ANCHE LA OVERLAP!!




##DB CON TUTTI ELEMENTI

In [ ]:
AnnotazioneMedCAT.columns= ['attribute', 'detected_name', 'pretty_name', 'context_similarity']

In [ ]:
# aggiungo anche nome tabella e nome attributo
AnnotazioneMedCAT = AnnotazioneMedCAT.copy()

# Separiamo la colonna 'attribute' in base all'ultimo '_', separando in 3 parti
split_columns = AnnotazioneMedCAT['attribute'].str.rsplit('_', n=2, expand=True)

# Usa .loc per assegnare le nuove colonne al DataFrame originale
AnnotazioneMedCAT.loc[:, 'Tabella'] = split_columns[0]
AnnotazioneMedCAT.loc[:, 'Attributo'] = split_columns[1] + '_' + split_columns[2]

# Visualizza il risultato
AnnotazioneMedCAT

,attribute,detected_name,pretty_name,context_similarity,Tabella,Attributo
0,beneficiarysummary_bene_birth_dt,date~of~birth,Date of birth,1.00,beneficiarysummary_bene,birth_dt
1,beneficiarysummary_bene_death_dt,date~of~death,Date of death,1.00,beneficiarysummary_bene,death_dt
2,beneficiarysummary_bene_esrd_ind,end~stage~renal~disease,End-stage renal disease,1.00,beneficiarysummary_bene,esrd_ind
3,beneficiarysummary_sp_state_code,state,State,1.00,beneficiarysummary_sp,state_code
4,beneficiarysummary_bene_hi_cvrage_tot_mons,total,Total,0.99,beneficiarysummary_bene_hi_cvrage,tot_mons
...,...,...,...,...,...,...
5912,providers_state,abbreviation,Abbreviation,1.00,providers,NaN
5913,providers_zip,patient,Patient,1.00,providers,NaN
5914,providers_zip,street~address,Street address,1.00,providers,NaN
5915,providers_utilization,patient,Patient,1.00,providers,NaN


il dataframe GMT ha come columns ['SOURCE', 'GAT', 'LAT']
dove GAT corrisponde ad attribute AnnotazioneMedCAT con l'unica differenza che mentre in GMT
ho un GAT ad esempio beneficiarysummary_bene_birth_dt
in AnnotazioneMedCAT me lo trovo riscritto come beneficiarysummary-bene_birth_dt
cioè il nome della tabella è separato da _ nel primo e da - nel secondo:
quindi prima di fare join devo uniformare

La GMT mantiene le corrispondenze di ogni singolo schema delle tre sorgenti rispetto allo schema OMOP
(siamo nella situazione top-down)


In [ ]:
TuttiGliSchemi.columns

Index(['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo'], dtype='object')

In [ ]:
# Eseguiamo il merge con AnnotazioneMedCAT
merged_df = TuttiGliSchemi.merge(AnnotazioneMedCAT, left_on='AttributoNomeCompleto', right_on='attribute', how='inner')

# Visualizziamo i primi risultati
merged_df.head()

NewAnnotazioneMedCAT=merged_df[['SOURCE', 'AttributoNomeCompleto', 'Tabella_x', 'Attributo_x', 'detected_name','pretty_name', 'context_similarity' ]]

In [ ]:
NewAnnotazioneMedCAT.head()
AnnotazioneMedCAT[AnnotazioneMedCAT['attribute'].isin(NewAnnotazioneMedCAT['AttributoNomeCompleto'])]

KeyError: 'attribute'

In [ ]:
NewAnnotazioneMedCAT[['SOURCE']].drop_duplicates()

,SOURCE
0,Synthea
398,Csm
631,Mimic
1912,OMOP


In [ ]:
NewAnnotazioneMedCAT.columns=['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo',
       'detected_name', 'pretty_name', 'context_similarity']

In [ ]:
# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Mimic']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

In [ ]:
omop_data

,SOURCE,AttributoNomeCompleto,Tabella,Attributo,detected_name,pretty_name,context_similarity
1912,OMOP,person_person_id,person,person_id,person,Person,1.00
1913,OMOP,person_person_id,person,person_id,domain,Concept model domain,1.00
1914,OMOP,person_person_id,person,person_id,contain,Containing,1.00
1915,OMOP,person_person_id,person,person_id,identify,Identified,1.00
1916,OMOP,person_person_id,person,person_id,patient,Patient,1.00
...,...,...,...,...,...,...,...
5921,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,nurse,Professional nurse,1.00
5922,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,midwife,Registered midwife,1.00
5923,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,physical~therapist,Physiotherapist,1.00
5924,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,refer,Patient referral,1.00


In [ ]:
mimic_data

,SOURCE,AttributoNomeCompleto,Tabella,Attributo,detected_name,pretty_name,context_similarity
631,Mimic,admissions_mimic_id,admissions,mimic_id,admission,Hospital admission,1.0
632,Mimic,admissions_mimic_id,admissions,mimic_id,table,"Table, device",1.0
633,Mimic,admissions_mimic_id,admissions,mimic_id,give~information,Informing,1.0
634,Mimic,admissions_mimic_id,admissions,mimic_id,patient,Patient,1.0
635,Mimic,admissions_mimic_id,admissions,mimic_id,admission,Hospital admission,1.0
...,...,...,...,...,...,...,...
1907,Mimic,transfers_mimic_id,transfers,mimic_id,physical,Physical,1.0
1908,Mimic,transfers_mimic_id,transfers,mimic_id,location,Location,1.0
1909,Mimic,transfers_mimic_id,transfers,mimic_id,patient,Patient,1.0
1910,Mimic,transfers_mimic_id,transfers,mimic_id,hospital,Hospital,1.0


In [ ]:
from collections import defaultdict
import pandas as pd

# Definisci una soglia di similarità per OSS
oss_threshold = 0.5  # Puoi modificarla in base alle tue necessità

# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Mimic']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute_omop.get(attribute1, set())
    P2 = pretty_name_by_attribute_mimic.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))

    return intersection

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA) di OMOP
top_oss_results = defaultdict(list)
no_match_results = []  # Lista per gli attributi che non hanno corrispondenza

# Calcolare OSS tra tutte le combinazioni di attributi di OMOP e MIMIC
for (tableA, attributeA) in pretty_name_by_attribute_omop.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute_mimic.keys():
        oss = calculate_oss((tableA, attributeA), (tableB, attributeB))

        if oss >= oss_threshold:
            oss_list.append(((tableB, attributeB), oss))

    # Se ci sono corrispondenze sopra la soglia, prendi le prime 3
    if oss_list:
        top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]
    else:
        # Se non ci sono corrispondenze sopra la soglia, aggiungi "assenza"
        no_match_results.append((tableA, attributeA, "Assente in MIMIC", "Assente in MIMIC", 0.0))

# Aggiungi i risultati di "assenza" al dizionario finale
df_no_match = pd.DataFrame(no_match_results, columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Convertiamo i risultati delle corrispondenze in un DataFrame
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Unisci i risultati di corrispondenza con quelli di "assenza"
df_final_result = pd.concat([df_top_oss, df_no_match], ignore_index=True)

# Ordina i risultati in base all'OSS in ordine decrescente
df_final_result = df_final_result.sort_values(by='OSS', ascending=False)

# Visualizza i risultati
df_final_result.head(10)  # Mostra i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
36,note,visit_detail_id,inputevents_cv,mimic_id,0.666667
13,cohort_definition,definition_type_concept_id,d_labitems,mimic_id,0.666667
33,note,provider_id,inputevents_cv,mimic_id,0.666667
34,note,provider_id,inputevents_mv,mimic_id,0.666667
35,note,provider_id,outputevents,mimic_id,0.666667
37,note,visit_detail_id,inputevents_mv,mimic_id,0.666667
20,note,note_date,outputevents,mimic_id,0.666667
19,note,note_date,inputevents_mv,mimic_id,0.666667
18,note,note_date,inputevents_cv,mimic_id,0.666667
38,note,visit_detail_id,outputevents,mimic_id,0.666667


In [ ]:
from collections import defaultdict
import pandas as pd

# Definisci una soglia di similarità per OSS
oss_threshold = 0.1  # Puoi modificarla in base alle tue necessità

# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Synthea']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute_omop.get(attribute1, set())
    P2 = pretty_name_by_attribute_mimic.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))

    return intersection

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA) di OMOP
top_oss_results = defaultdict(list)

# Calcolare OSS tra tutte le combinazioni di attributi di OMOP e MIMIC
for (tableA, attributeA) in pretty_name_by_attribute_omop.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute_mimic.keys():
        oss = calculate_oss((tableA, attributeA), (tableB, attributeB))

        if oss >= oss_threshold:
            oss_list.append(((tableB, attributeB), oss))

    # Se ci sono corrispondenze sopra la soglia, prendi le prime 3
    if oss_list:
        top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]

# Convertiamo i risultati in un DataFrame per una visualizzazione più chiara
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Ordina i risultati in base all'OSS in ordine decrescente
df_top_oss = df_top_oss.sort_values(by='OSS', ascending=False)

# Mostra i risultati solo per quelli che superano la soglia
df_top_oss  # Mostra i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
147,note,provider_id,patients,birthdate,0.666667
153,note,visit_occurrence_id,patients,birthdate,0.666667
117,note,note_date,patients,birthdate,0.666667
150,note,visit_detail_id,patients,birthdate,0.666667
149,note,provider_id,immunizations,date,0.500000
...,...,...,...,...,...
18,cohort_definition,cohort_definition_name,patients,gender,0.100000
177,payer_plan_period,person_id,patients,ssn,0.100000
178,payer_plan_period,plan_source_value,encounters,reasoncode,0.100000
214,person,provider_id,allergies,description,0.100000


In [ ]:
from collections import defaultdict

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo)
pretty_name_by_attribute = AnnotazioneMedCAT.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute.get(attribute1, set())
    P2 = pretty_name_by_attribute.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))

    return intersection

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA)
top_oss_results = defaultdict(list)

# Calcolare OSS tra tutte le combinazioni di attributi
for (tableA, attributeA) in pretty_name_by_attribute.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute.keys():
        # Assicuriamoci che la TabellaA e TabellaB siano diverse
        if tableA != tableB:
            oss = calculate_oss((tableA, attributeA), (tableB, attributeB))
            oss_list.append(((tableB, attributeB), oss))

    # Ordiniamo per similarità OSS in ordine decrescente e prendiamo i primi 3
    top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]

# Convertiamo i risultati in DataFrame per una visualizzazione più chiara
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Mostriamo i risultati
df_top_oss.head(10)  # Visualizza i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
0,admissions,admission_location,admissions_hospital,expire_flag,0.454545
1,admissions,admission_location,transfers,mimic_id,0.400000
2,admissions,admission_location,transfers,prev_wardid,0.333333
3,admissions,admission_type,admissions_hospital,expire_flag,0.625000
4,admissions,admission_type,services,mimic_id,0.428571
5,admissions,admission_type,inputevents_cv,mimic_id,0.400000
6,admissions,hadm_id,admissions_hospital,expire_flag,0.625000
7,admissions,hadm_id,services,mimic_id,0.428571
8,admissions,hadm_id,inputevents_cv,mimic_id,0.400000
9,admissions,marital_status,admissions_hospital,expire_flag,0.555556


##DF CON CONTEXT SIMILARITY = 1

In [ ]:
AnnotazioneMedCAT_MAX.columns= ['attribute', 'detected_name', 'pretty_name', 'context_similarity']

In [ ]:
# aggiungo anche nome tabella e nome attributo
AnnotazioneMedCAT_MAX = AnnotazioneMedCAT_MAX.copy()

# Separiamo la colonna 'attribute' in base all'ultimo '_', separando in 3 parti
split_columns = AnnotazioneMedCAT_MAX['attribute'].str.rsplit('_', n=2, expand=True)

# Usa .loc per assegnare le nuove colonne al DataFrame originale
AnnotazioneMedCAT_MAX.loc[:, 'Tabella'] = split_columns[0]
AnnotazioneMedCAT_MAX.loc[:, 'Attributo'] = split_columns[1] + '_' + split_columns[2]

# Visualizza il risultato
AnnotazioneMedCAT_MAX

,attribute,detected_name,pretty_name,context_similarity,Tabella,Attributo
0,beneficiarysummary_bene_birth_dt,date~of~birth,Date of birth,1.0,beneficiarysummary_bene,birth_dt
1,beneficiarysummary_bene_death_dt,date~of~death,Date of death,1.0,beneficiarysummary_bene,death_dt
2,beneficiarysummary_bene_esrd_ind,end~stage~renal~disease,End-stage renal disease,1.0,beneficiarysummary_bene,esrd_ind
3,beneficiarysummary_sp_state_code,state,State,1.0,beneficiarysummary_sp,state_code
5,beneficiarysummary_bene_hi_cvrage_tot_mons,of~part,In (attribute),1.0,beneficiarysummary_bene_hi_cvrage,tot_mons
...,...,...,...,...,...,...
5912,providers_state,abbreviation,Abbreviation,1.0,providers,NaN
5913,providers_zip,patient,Patient,1.0,providers,NaN
5914,providers_zip,street~address,Street address,1.0,providers,NaN
5915,providers_utilization,patient,Patient,1.0,providers,NaN


il dataframe GMT ha come columns ['SOURCE', 'GAT', 'LAT']
dove GAT corrisponde ad attribute AnnotazioneMedCAT con l'unica differenza che mentre in GMT
ho un GAT ad esempio beneficiarysummary_bene_birth_dt
in AnnotazioneMedCAT me lo trovo riscritto come beneficiarysummary-bene_birth_dt
cioè il nome della tabella è separato da _ nel primo e da - nel secondo:
quindi prima di fare join devo uniformare

La GMT mantiene le corrispondenze di ogni singolo schema delle tre sorgenti rispetto allo schema OMOP
(siamo nella situazione top-down)


In [ ]:
TuttiGliSchemi.columns

Index(['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo'], dtype='object')

In [ ]:
# Eseguiamo il merge con AnnotazioneMedCAT
merged_df = TuttiGliSchemi.merge(AnnotazioneMedCAT_MAX, left_on='AttributoNomeCompleto', right_on='attribute', how='inner')

# Visualizziamo i primi risultati
merged_df.head()

NewAnnotazioneMedCAT=merged_df[['SOURCE', 'AttributoNomeCompleto', 'Tabella_x', 'Attributo_x', 'detected_name','pretty_name', 'context_similarity' ]]

In [ ]:
NewAnnotazioneMedCAT.head()
AnnotazioneMedCAT_MAX[AnnotazioneMedCAT_MAX['attribute'].isin(NewAnnotazioneMedCAT['AttributoNomeCompleto'])]

,attribute,detected_name,pretty_name,context_similarity,Tabella,Attributo
0,beneficiarysummary_bene_birth_dt,date~of~birth,Date of birth,1.0,beneficiarysummary_bene,birth_dt
1,beneficiarysummary_bene_death_dt,date~of~death,Date of death,1.0,beneficiarysummary_bene,death_dt
2,beneficiarysummary_bene_esrd_ind,end~stage~renal~disease,End-stage renal disease,1.0,beneficiarysummary_bene,esrd_ind
3,beneficiarysummary_sp_state_code,state,State,1.0,beneficiarysummary_sp,state_code
5,beneficiarysummary_bene_hi_cvrage_tot_mons,of~part,In (attribute),1.0,beneficiarysummary_bene_hi_cvrage,tot_mons
...,...,...,...,...,...,...
5912,providers_state,abbreviation,Abbreviation,1.0,providers,NaN
5913,providers_zip,patient,Patient,1.0,providers,NaN
5914,providers_zip,street~address,Street address,1.0,providers,NaN
5915,providers_utilization,patient,Patient,1.0,providers,NaN


In [ ]:
NewAnnotazioneMedCAT[['SOURCE']].drop_duplicates()

,SOURCE
0,Synthea
331,Csm
525,Mimic
1556,OMOP


In [ ]:
NewAnnotazioneMedCAT.columns=['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo',
       'detected_name', 'pretty_name', 'context_similarity']

In [ ]:
# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Mimic']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

In [ ]:
omop_data

,SOURCE,AttributoNomeCompleto,Tabella,Attributo,detected_name,pretty_name,context_similarity
1556,OMOP,person_person_id,person,person_id,person,Person,1.0
1557,OMOP,person_person_id,person,person_id,domain,Concept model domain,1.0
1558,OMOP,person_person_id,person,person_id,contain,Containing,1.0
1559,OMOP,person_person_id,person,person_id,identify,Identified,1.0
1560,OMOP,person_person_id,person,person_id,patient,Patient,1.0
...,...,...,...,...,...,...,...
4677,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,physician,Physician,1.0
4678,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,nurse,Professional nurse,1.0
4679,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,midwife,Registered midwife,1.0
4680,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,physical~therapist,Physiotherapist,1.0


In [ ]:
mimic_data

,SOURCE,AttributoNomeCompleto,Tabella,Attributo,detected_name,pretty_name,context_similarity
525,Mimic,admissions_mimic_id,admissions,mimic_id,admission,Hospital admission,1.0
526,Mimic,admissions_mimic_id,admissions,mimic_id,table,"Table, device",1.0
527,Mimic,admissions_mimic_id,admissions,mimic_id,give~information,Informing,1.0
528,Mimic,admissions_mimic_id,admissions,mimic_id,patient,Patient,1.0
529,Mimic,admissions_mimic_id,admissions,mimic_id,admission,Hospital admission,1.0
...,...,...,...,...,...,...,...
1551,Mimic,transfers_mimic_id,transfers,mimic_id,physical,Physical,1.0
1552,Mimic,transfers_mimic_id,transfers,mimic_id,location,Location,1.0
1553,Mimic,transfers_mimic_id,transfers,mimic_id,patient,Patient,1.0
1554,Mimic,transfers_mimic_id,transfers,mimic_id,hospital,Hospital,1.0


In [ ]:
from collections import defaultdict
import pandas as pd

# Definisci una soglia di similarità per OSS
oss_threshold = 0.5  # Puoi modificarla in base alle tue necessità

# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Mimic']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute_omop.get(attribute1, set())
    P2 = pretty_name_by_attribute_mimic.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))

    return intersection

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA) di OMOP
top_oss_results = defaultdict(list)
no_match_results = []  # Lista per gli attributi che non hanno corrispondenza

# Calcolare OSS tra tutte le combinazioni di attributi di OMOP e MIMIC
for (tableA, attributeA) in pretty_name_by_attribute_omop.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute_mimic.keys():
        oss = calculate_oss((tableA, attributeA), (tableB, attributeB))

        if oss >= oss_threshold:
            oss_list.append(((tableB, attributeB), oss))

    # Se ci sono corrispondenze sopra la soglia, prendi le prime 3
    if oss_list:
        top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]
    else:
        # Se non ci sono corrispondenze sopra la soglia, aggiungi "assenza"
        no_match_results.append((tableA, attributeA, "Assente in MIMIC", "Assente in MIMIC", 0.0))

# Aggiungi i risultati di "assenza" al dizionario finale
df_no_match = pd.DataFrame(no_match_results, columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Convertiamo i risultati delle corrispondenze in un DataFrame
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Unisci i risultati di corrispondenza con quelli di "assenza"
df_final_result = pd.concat([df_top_oss, df_no_match], ignore_index=True)

# Ordina i risultati in base all'OSS in ordine decrescente
df_final_result = df_final_result.sort_values(by='OSS', ascending=False)

# Visualizza i risultati
df_final_result.head(10)  # Mostra i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
45,note,note_date,outputevents,mimic_id,1.0
43,note,note_date,inputevents_cv,mimic_id,1.0
73,note,visit_detail_id,inputevents_cv,mimic_id,1.0
64,note,note_title,inputevents_cv,mimic_id,1.0
75,note,visit_detail_id,outputevents,mimic_id,1.0
65,note,note_title,inputevents_mv,mimic_id,1.0
72,note,provider_id,outputevents,mimic_id,1.0
71,note,provider_id,inputevents_mv,mimic_id,1.0
76,note,visit_occurrence_id,inputevents_cv,mimic_id,1.0
77,note,visit_occurrence_id,inputevents_mv,mimic_id,1.0


In [ ]:
from collections import defaultdict
import pandas as pd

# Definisci una soglia di similarità per OSS
oss_threshold = 0.1  # Puoi modificarla in base alle tue necessità

# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Synthea']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute_omop.get(attribute1, set())
    P2 = pretty_name_by_attribute_mimic.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))

    return intersection

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA) di OMOP
top_oss_results = defaultdict(list)

# Calcolare OSS tra tutte le combinazioni di attributi di OMOP e MIMIC
for (tableA, attributeA) in pretty_name_by_attribute_omop.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute_mimic.keys():
        oss = calculate_oss((tableA, attributeA), (tableB, attributeB))

        if oss >= oss_threshold:
            oss_list.append(((tableB, attributeB), oss))

    # Se ci sono corrispondenze sopra la soglia, prendi le prime 3
    if oss_list:
        top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]

# Convertiamo i risultati in un DataFrame per una visualizzazione più chiara
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Ordina i risultati in base all'OSS in ordine decrescente
df_top_oss = df_top_oss.sort_values(by='OSS', ascending=False)

# Mostra i risultati solo per quelli che superano la soglia
df_top_oss  # Mostra i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
138,note,note_id,patients,id,0.666667
139,note,note_id,patients,passport,0.666667
128,note,note_datetime,encounters,stop,0.666667
137,note,note_id,encounters,id,0.666667
155,note,provider_id,allergies,code,0.500000
...,...,...,...,...,...
97,drug_exposure,drug_exposure_id,careplans,id,0.100000
96,death,person_id,patients,passport,0.100000
95,death,person_id,patients,id,0.100000
94,death,person_id,encounters,id,0.100000


In [ ]:
from collections import defaultdict

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo)
pretty_name_by_attribute = AnnotazioneMedCAT_MAX.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute.get(attribute1, set())
    P2 = pretty_name_by_attribute.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))

    return intersection

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA)
top_oss_results = defaultdict(list)

# Calcolare OSS tra tutte le combinazioni di attributi
for (tableA, attributeA) in pretty_name_by_attribute.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute.keys():
        # Assicuriamoci che la TabellaA e TabellaB siano diverse
        if tableA != tableB:
            oss = calculate_oss((tableA, attributeA), (tableB, attributeB))
            oss_list.append(((tableB, attributeB), oss))

    # Ordiniamo per similarità OSS in ordine decrescente e prendiamo i primi 3
    top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]

# Convertiamo i risultati in DataFrame per una visualizzazione più chiara
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Mostriamo i risultati
df_top_oss.head(10)  # Visualizza i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
0,admissions,admission_location,admissions_hospital,expire_flag,0.555556
1,admissions,admission_location,transfers,mimic_id,0.400000
2,admissions,admission_location,transfers,prev_wardid,0.363636
3,admissions,admission_type,admissions_hospital,expire_flag,0.833333
4,admissions,admission_type,services,mimic_id,0.428571
5,admissions,admission_type,inputevents_cv,mimic_id,0.400000
6,admissions,hadm_id,admissions_hospital,expire_flag,0.833333
7,admissions,hadm_id,services,mimic_id,0.428571
8,admissions,hadm_id,inputevents_cv,mimic_id,0.400000
9,admissions,marital_status,admissions_hospital,expire_flag,0.714286


##DF CON CONTEXT SIMILARITY = 0.8

In [ ]:
AnnotazioneMedCAT_08.columns= ['attribute', 'detected_name', 'pretty_name', 'context_similarity']

In [ ]:
# aggiungo anche nome tabella e nome attributo
AnnotazioneMedCAT_08 = AnnotazioneMedCAT_08.copy()

# Separiamo la colonna 'attribute' in base all'ultimo '_', separando in 3 parti
split_columns = AnnotazioneMedCAT_08['attribute'].str.rsplit('_', n=2, expand=True)

# Usa .loc per assegnare le nuove colonne al DataFrame originale
AnnotazioneMedCAT_08.loc[:, 'Tabella'] = split_columns[0]
AnnotazioneMedCAT_08.loc[:, 'Attributo'] = split_columns[1] + '_' + split_columns[2]

# Visualizza il risultato
AnnotazioneMedCAT_08

,attribute,detected_name,pretty_name,context_similarity,Tabella,Attributo
0,beneficiarysummary_bene_birth_dt,date~of~birth,Date of birth,1.00,beneficiarysummary_bene,birth_dt
1,beneficiarysummary_bene_death_dt,date~of~death,Date of death,1.00,beneficiarysummary_bene,death_dt
2,beneficiarysummary_bene_esrd_ind,end~stage~renal~disease,End-stage renal disease,1.00,beneficiarysummary_bene,esrd_ind
3,beneficiarysummary_sp_state_code,state,State,1.00,beneficiarysummary_sp,state_code
4,beneficiarysummary_bene_hi_cvrage_tot_mons,total,Total,0.99,beneficiarysummary_bene_hi_cvrage,tot_mons
...,...,...,...,...,...,...
5912,providers_state,abbreviation,Abbreviation,1.00,providers,NaN
5913,providers_zip,patient,Patient,1.00,providers,NaN
5914,providers_zip,street~address,Street address,1.00,providers,NaN
5915,providers_utilization,patient,Patient,1.00,providers,NaN


il dataframe GMT ha come columns ['SOURCE', 'GAT', 'LAT']
dove GAT corrisponde ad attribute AnnotazioneMedCAT con l'unica differenza che mentre in GMT
ho un GAT ad esempio beneficiarysummary_bene_birth_dt
in AnnotazioneMedCAT me lo trovo riscritto come beneficiarysummary-bene_birth_dt
cioè il nome della tabella è separato da _ nel primo e da - nel secondo:
quindi prima di fare join devo uniformare

La GMT mantiene le corrispondenze di ogni singolo schema delle tre sorgenti rispetto allo schema OMOP
(siamo nella situazione top-down)


In [ ]:
TuttiGliSchemi.columns

Index(['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo'], dtype='object')

In [ ]:
# Eseguiamo il merge con AnnotazioneMedCAT
merged_df = TuttiGliSchemi.merge(AnnotazioneMedCAT_08, left_on='AttributoNomeCompleto', right_on='attribute', how='inner')

# Visualizziamo i primi risultati
merged_df.head()

NewAnnotazioneMedCAT=merged_df[['SOURCE', 'AttributoNomeCompleto', 'Tabella_x', 'Attributo_x', 'detected_name','pretty_name', 'context_similarity' ]]

In [ ]:
NewAnnotazioneMedCAT.head()
AnnotazioneMedCAT_08[AnnotazioneMedCAT_08['attribute'].isin(NewAnnotazioneMedCAT['AttributoNomeCompleto'])]

,attribute,detected_name,pretty_name,context_similarity,Tabella,Attributo
0,beneficiarysummary_bene_birth_dt,date~of~birth,Date of birth,1.00,beneficiarysummary_bene,birth_dt
1,beneficiarysummary_bene_death_dt,date~of~death,Date of death,1.00,beneficiarysummary_bene,death_dt
2,beneficiarysummary_bene_esrd_ind,end~stage~renal~disease,End-stage renal disease,1.00,beneficiarysummary_bene,esrd_ind
3,beneficiarysummary_sp_state_code,state,State,1.00,beneficiarysummary_sp,state_code
4,beneficiarysummary_bene_hi_cvrage_tot_mons,total,Total,0.99,beneficiarysummary_bene_hi_cvrage,tot_mons
...,...,...,...,...,...,...
5912,providers_state,abbreviation,Abbreviation,1.00,providers,NaN
5913,providers_zip,patient,Patient,1.00,providers,NaN
5914,providers_zip,street~address,Street address,1.00,providers,NaN
5915,providers_utilization,patient,Patient,1.00,providers,NaN


In [ ]:
NewAnnotazioneMedCAT[['SOURCE']].drop_duplicates()

,SOURCE
0,Synthea
358,Csm
560,Mimic
1647,OMOP


In [ ]:
NewAnnotazioneMedCAT.columns=['SOURCE', 'AttributoNomeCompleto', 'Tabella', 'Attributo',
       'detected_name', 'pretty_name', 'context_similarity']

In [ ]:
# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Mimic']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

In [ ]:
omop_data

,SOURCE,AttributoNomeCompleto,Tabella,Attributo,detected_name,pretty_name,context_similarity
1647,OMOP,person_person_id,person,person_id,person,Person,1.00
1648,OMOP,person_person_id,person,person_id,domain,Concept model domain,1.00
1649,OMOP,person_person_id,person,person_id,contain,Containing,1.00
1650,OMOP,person_person_id,person,person_id,identify,Identified,1.00
1651,OMOP,person_person_id,person,person_id,patient,Patient,1.00
...,...,...,...,...,...,...,...
5042,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,nurse,Professional nurse,1.00
5043,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,midwife,Registered midwife,1.00
5044,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,physical~therapist,Physiotherapist,1.00
5045,OMOP,provider_gender_source_concept_id,provider,gender_source_concept_id,refer,Patient referral,1.00


In [ ]:
mimic_data

,SOURCE,AttributoNomeCompleto,Tabella,Attributo,detected_name,pretty_name,context_similarity
560,Mimic,admissions_mimic_id,admissions,mimic_id,admission,Hospital admission,1.0
561,Mimic,admissions_mimic_id,admissions,mimic_id,table,"Table, device",1.0
562,Mimic,admissions_mimic_id,admissions,mimic_id,give~information,Informing,1.0
563,Mimic,admissions_mimic_id,admissions,mimic_id,patient,Patient,1.0
564,Mimic,admissions_mimic_id,admissions,mimic_id,admission,Hospital admission,1.0
...,...,...,...,...,...,...,...
1642,Mimic,transfers_mimic_id,transfers,mimic_id,physical,Physical,1.0
1643,Mimic,transfers_mimic_id,transfers,mimic_id,location,Location,1.0
1644,Mimic,transfers_mimic_id,transfers,mimic_id,patient,Patient,1.0
1645,Mimic,transfers_mimic_id,transfers,mimic_id,hospital,Hospital,1.0


In [ ]:
from collections import defaultdict
import pandas as pd

# Definisci una soglia di similarità per OSS
oss_threshold = 0.5  # Puoi modificarla in base alle tue necessità

# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Mimic']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute_omop.get(attribute1, set())
    P2 = pretty_name_by_attribute_mimic.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))

    return intersection n

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA) di OMOP
top_oss_results = defaultdict(list)
no_match_results = []  # Lista per gli attributi che non hanno corrispondenza

# Calcolare OSS tra tutte le combinazioni di attributi di OMOP e MIMIC
for (tableA, attributeA) in pretty_name_by_attribute_omop.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute_mimic.keys():
        oss = calculate_oss((tableA, attributeA), (tableB, attributeB))

        if oss >= oss_threshold:
            oss_list.append(((tableB, attributeB), oss))

    # Se ci sono corrispondenze sopra la soglia, prendi le prime 3
    if oss_list:
        top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]
    else:
        # Se non ci sono corrispondenze sopra la soglia, aggiungi "assenza"
        no_match_results.append((tableA, attributeA, "Assente in MIMIC", "Assente in MIMIC", 0.0))

# Aggiungi i risultati di "assenza" al dizionario finale
df_no_match = pd.DataFrame(no_match_results, columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Convertiamo i risultati delle corrispondenze in un DataFrame
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Unisci i risultati di corrispondenza con quelli di "assenza"
df_final_result = pd.concat([df_top_oss, df_no_match], ignore_index=True)

# Ordina i risultati in base all'OSS in ordine decrescente
df_final_result = df_final_result.sort_values(by='OSS', ascending=False)

# Visualizza i risultati
df_final_result.head(10)  # Mostra i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
65,note,visit_detail_id,inputevents_cv,mimic_id,1.0
70,note,visit_occurrence_id,outputevents,mimic_id,1.0
69,note,visit_occurrence_id,inputevents_mv,mimic_id,1.0
68,note,visit_occurrence_id,inputevents_cv,mimic_id,1.0
67,note,visit_detail_id,outputevents,mimic_id,1.0
37,note,note_date,outputevents,mimic_id,1.0
36,note,note_date,inputevents_mv,mimic_id,1.0
66,note,visit_detail_id,inputevents_mv,mimic_id,1.0
35,note,note_date,inputevents_cv,mimic_id,1.0
64,note,provider_id,outputevents,mimic_id,1.0


In [ ]:
from collections import defaultdict
import pandas as pd

# Definisci una soglia di similarità per OSS
oss_threshold = 0.1  # Puoi modificarla in base alle tue necessità

# Filtriamo i dati per la SOURCE='OMOP' e 'MIMIC'
omop_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'OMOP']
mimic_data = NewAnnotazioneMedCAT[NewAnnotazioneMedCAT['SOURCE'] == 'Synthea']

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo) per OMOP e MIMIC
pretty_name_by_attribute_omop = omop_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()
pretty_name_by_attribute_mimic = mimic_data.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute_omop.get(attribute1, set())
    P2 = pretty_name_by_attribute_mimic.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))

    return intersection

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA) di OMOP
top_oss_results = defaultdict(list)

# Calcolare OSS tra tutte le combinazioni di attributi di OMOP e MIMIC
for (tableA, attributeA) in pretty_name_by_attribute_omop.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute_mimic.keys():
        oss = calculate_oss((tableA, attributeA), (tableB, attributeB))

        if oss >= oss_threshold:
            oss_list.append(((tableB, attributeB), oss))

    # Se ci sono corrispondenze sopra la soglia, prendi le prime 3
    if oss_list:
        top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]

# Convertiamo i risultati in un DataFrame per una visualizzazione più chiara
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Ordina i risultati in base all'OSS in ordine decrescente
df_top_oss = df_top_oss.sort_values(by='OSS', ascending=False)

# Mostra i risultati solo per quelli che superano la soglia
df_top_oss  # Mostra i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
133,note,note_id,patients,passport,0.666667
132,note,note_id,patients,id,0.666667
131,note,note_id,encounters,id,0.666667
122,note,note_datetime,encounters,stop,0.666667
130,note,note_event_id,patients,passport,0.500000
...,...,...,...,...,...
183,payer_plan_period,person_id,patients,id,0.100000
182,payer_plan_period,person_id,encounters,id,0.100000
181,payer_plan_period,payer_source_value,encounters,reasoncode,0.100000
303,specimen,disease_status_concept_id,patients,id,0.100000


In [ ]:
from collections import defaultdict

# Creiamo una lista di set di pretty_name per ogni coppia (Tabella, Attributo)
pretty_name_by_attribute = AnnotazioneMedCAT_08.groupby(['Tabella', 'Attributo'])['pretty_name'].apply(set).to_dict()

# Funzione per calcolare Overlap Set Similarity (OSS) tra due attributi
def calculate_oss(attribute1, attribute2):
    P1 = pretty_name_by_attribute.get(attribute1, set())
    P2 = pretty_name_by_attribute.get(attribute2, set())

    # Calcolo OSS
    intersection = len(P1.intersection(P2))

    return intersection

# Dizionario per raccogliere le migliori 3 corrispondenze per ogni (TabellaA, AttributoA)
top_oss_results = defaultdict(list)

# Calcolare OSS tra tutte le combinazioni di attributi
for (tableA, attributeA) in pretty_name_by_attribute.keys():
    oss_list = []

    for (tableB, attributeB) in pretty_name_by_attribute.keys():
        # Assicuriamoci che la TabellaA e TabellaB siano diverse
        if tableA != tableB:
            oss = calculate_oss((tableA, attributeA), (tableB, attributeB))
            oss_list.append(((tableB, attributeB), oss))

    # Ordiniamo per similarità OSS in ordine decrescente e prendiamo i primi 3
    top_oss_results[(tableA, attributeA)] = sorted(oss_list, key=lambda x: x[1], reverse=True)[:3]

# Convertiamo i risultati in DataFrame per una visualizzazione più chiara
df_top_oss = pd.DataFrame([
    (tableA, attributeA, tableB, attributeB, oss)
    for (tableA, attributeA), matches in top_oss_results.items()
    for (tableB, attributeB), oss in matches
], columns=['TabellaA', 'AttributoA', 'TabellaB', 'AttributoB', 'OSS'])

# Mostriamo i risultati
df_top_oss.head(10)  # Visualizza i primi 10 risultati

,TabellaA,AttributoA,TabellaB,AttributoB,OSS
0,admissions,admission_location,admissions_hospital,expire_flag,0.555556
1,admissions,admission_location,transfers,mimic_id,0.400000
2,admissions,admission_location,transfers,prev_wardid,0.363636
3,admissions,admission_type,admissions_hospital,expire_flag,0.833333
4,admissions,admission_type,services,mimic_id,0.428571
5,admissions,admission_type,inputevents_cv,mimic_id,0.400000
6,admissions,hadm_id,admissions_hospital,expire_flag,0.833333
7,admissions,hadm_id,services,mimic_id,0.428571
8,admissions,hadm_id,inputevents_cv,mimic_id,0.400000
9,admissions,marital_status,admissions_hospital,expire_flag,0.714286


# NON CONSIDERARE -MOMIS ANNOTATOR

#### 1) Controllare in MOMIS ANNOTATOR il vocabolario CDM

In sostanza capire quali sono le glosse (e altre informazioni disponibili) per gli attributi di ciascuna tabella CDM, e confrontarle con quelle qui disponibili.
Ad esempio per la tabella Observation abbiamo


In [ ]:
TABELLE_ATTRIBUTI_OMOP[TABELLE_ATTRIBUTI_OMOP['omop'].str.contains(f'^{"observation"}-')].sample(2)

,omop,des1,d1,d2
15429,observation-observation_datetime,"the observation table captures clinical facts about a person obtained in the context of examination, questioning or a procedure. any data that cannot be represented by any other domains, such as social and lifestyle facts, medical history, family history, etc. are recorded here.;the date and time of the observation.","the observation table captures clinical facts about a person obtained in the context of examination, questioning or a procedure. any data that cannot be represented by any other domains, such as social and lifestyle facts, medical history, family history, etc. are recorded here.",the date and time of the observation.
16206,observation-provider_id,"the observation table captures clinical facts about a person obtained in the context of examination, questioning or a procedure. any data that cannot be represented by any other domains, such as social and lifestyle facts, medical history, family history, etc. are recorded here.;a foreign key to the provider in the provider table who was responsible for making the observation.","the observation table captures clinical facts about a person obtained in the context of examination, questioning or a procedure. any data that cannot be represented by any other domains, such as social and lifestyle facts, medical history, family history, etc. are recorded here.",a foreign key to the provider in the provider table who was responsible for making the observation.


#### 2) Decidere quale glossa informazione considerare

cioè solo la descrizione della colonna oppure la descrizione completa tabella colonna?

Questo è un punto fondamentale da discutere con Riccardo: stabilire quali sono le informazioni migliori da associare a CDM per far funzionare meglio il sistema.

Ad esempio: Nella descrizione di **observation-observation_source_value** si può pensare di aggiungere anche informazioni che ho ricavato da precedenti matchin, in questo caso possiamo aggiungere  gli attributi che abbiamo trovato in Synthea
allergies-code, conditions-code,
o anche le glosse associate in Synthea  a questi attributi:




In [ ]:
# ad esempio per  allergies-code cerco quello corrispondente in Synthea
TABELLE_ATTRIBUTI_LOCALI[TABELLE_ATTRIBUTI_LOCALI['table'].str.contains("allergies-code")]

,SOURCE,table,des2,d3,d4
4,Synthea,allergies-code,patient allergy data.;allergy code from snomed-ct,patient allergy data.,allergy code from snomed-ct


In [ ]:
# ad esempio per  allergies-code cerco quello corrispondente in CSM
TABELLE_ATTRIBUTI_LOCALI[TABELLE_ATTRIBUTI_LOCALI['table'].str.contains("inpatientclaims-icd9_prcdr_cd")]


,SOURCE,table,des2,d3,d4
53,Csm,inpatientclaims-icd9_prcdr_cd,inpatientclaims pertains to a synthetic inpatient claim;claim procedure code 1 – claim procedure code 6,inpatientclaims pertains to a synthetic inpatient claim,claim procedure code 1 – claim procedure code 6


In questo modo, se a allergies-code associo anche le glosse "allergy code from snomed-ct",
"claim procedure code 1 – claim procedure code 6"
forse (è da provare ...) riesco a fare meglio le prossime volte


In [ ]:
# per trovare il GAT mappato più volte
result = GMT.groupby('SOURCE')['GAT'].apply(lambda x: x.value_counts().idxmax()).reset_index(name='MostFrequentGAT')
result = pd.merge(result, GMT, how='left', left_on=['SOURCE', 'MostFrequentGAT'], right_on=['SOURCE', 'GAT'])
result = result[['SOURCE', 'MostFrequentGAT', 'LAT']]
result

,SOURCE,MostFrequentGAT,LAT
0,Csm,cohort_definition-cohort_definition_name,beneficiarysummary-bene_esrd_ind
1,Csm,cohort_definition-cohort_definition_name,beneficiarysummary-sp_alzhdmta
2,Csm,cohort_definition-cohort_definition_name,beneficiarysummary-sp_chf
3,Csm,cohort_definition-cohort_definition_name,beneficiarysummary-sp_chrnkidn
4,Csm,cohort_definition-cohort_definition_name,beneficiarysummary-sp_cncr
5,Csm,cohort_definition-cohort_definition_name,beneficiarysummary-sp_copd
6,Csm,cohort_definition-cohort_definition_name,beneficiarysummary-sp_depressn
7,Csm,cohort_definition-cohort_definition_name,beneficiarysummary-sp_diabetes
8,Csm,cohort_definition-cohort_definition_name,beneficiarysummary-sp_ischmcht
9,Csm,cohort_definition-cohort_definition_name,beneficiarysummary-sp_osteoprs


In [ ]:
# per trovare il LAT mappato più volte
result = GMT.groupby('SOURCE')['LAT'].apply(lambda x: x.value_counts().idxmax()).reset_index(name='MostFrequentLAT')
result = pd.merge(result, GMT, how='left', left_on=['SOURCE', 'MostFrequentLAT'], right_on=['SOURCE', 'LAT'])
result = result[['SOURCE', 'MostFrequentLAT', 'GAT']]
result

,SOURCE,MostFrequentLAT,GAT
0,Csm,carrierclaims-prf_physn_npi,visit_occurrence-provider_id
1,Csm,carrierclaims-prf_physn_npi,measurement-provider_id
2,Csm,carrierclaims-prf_physn_npi,procedure_occurrence-provider_id
3,Csm,carrierclaims-prf_physn_npi,condition_occurrence-provider_id
4,Csm,carrierclaims-prf_physn_npi,observation-provider_id
5,Csm,carrierclaims-prf_physn_npi,drug_exposure-provider_id
6,Csm,carrierclaims-prf_physn_npi,device_exposure-provider_id
7,Csm,carrierclaims-prf_physn_npi,provider-npi
8,Synthea,conditions-code,condition_occurrence-condition_concept_id
9,Synthea,conditions-code,condition_occurrence-condition_source_value
